In [2]:
import pandas as pd
pd.set_option('max_colwidth', 100)
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import confusion_matrix
import itertools
import scipy.stats as st
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.feature_selection import mutual_info_classif
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 500)
import rpy2
#import pingouin as pg

import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
import rpy2.ipython.html
rpy2.ipython.html.init_printing()
from rpy2.rinterface_lib.embedded import RRuntimeError

# import R's "utils" package
utils = importr('utils')

lavaan = importr('lavaan')
semTools = importr('semTools')

In [3]:
stringr = importr('stringr')

In [4]:
# importing stuff the right way
try:
    semTools_version = '.'.join([str(ii) for ii in list(ro.r('packageVersion("semTools")')[0])])
except RRuntimeError:
    semTools_version = ''
# There's a bug in the main semtools libarary, this install from my fork of semtools where I've fixed it.
if semTools_version != '0.5.6.933':
    ro.r("""
        library(devtools)
        install_github("shotgunosine/semTools/semTools@aa68e38f88d9dd881617d5ade429bde2b2a74e08")
        """
    )

In [5]:
# minimal lavaan example
ro.r("""
library("lavaan")
library("semTools")

# specify the model
HS.model <- ' visual  =~ x1 + x2 + x3      
              textual =~ x4 + x5 + x6
              speed   =~ x7 + x8 + x9 '

# fit the model
fit <- cfa(HS.model, data = HolzingerSwineford1939)

# display summary output
summary(fit, fit.measures = TRUE)
""")


0,header,$lavaan.v...
1,optim,$estimato...
2,data,$ngroups ...
3,test,$standard...
4,fit,...
5,pe,...


In [8]:
# lavaan example
ro.r("""
library("lavaan")
library("semTools")
## create 3-group data in lavaan example(cfa) data
HS <- lavaan::HolzingerSwineford1939
head(HS,3)
HS$ageGroup <- ifelse(HS$ageyr < 13, "preteen",
                      ifelse(HS$ageyr > 13, "teen", "thirteen"))
## specify and fit an appropriate null model for incremental fit indices
mod.null <- c(paste0("x", 1:9, " ~ c(T", 1:9, ", T", 1:9, ", T", 1:9, ")*1"),
              paste0("x", 1:9, " ~~ c(L", 1:9, ", L", 1:9, ", L", 1:9, ")*x", 1:9))
fit.null <- cfa(mod.null, data = HS, group = "ageGroup")

## fit target model with varying levels of measurement equivalence
mod.config <- '
visual  =~ x1 + x2 + x3
textual =~ x4 + x5 + x6
speed   =~ x7 + x8 + x9
'
fit.config <- cfa(mod.config, data = HS, std.lv = TRUE, group = "ageGroup")
fit.metric <- cfa(mod.config, data = HS, std.lv = TRUE, group = "ageGroup",
                  group.equal = "loadings")
fit.scalar <- cfa(mod.config, data = HS, std.lv = TRUE, group = "ageGroup",
                  group.equal = c("loadings","intercepts"))

####################### Permutation Method
## fit indices of interest for multiparameter omnibus test
myAFIs <- c("chisq","cfi","rmsea","mfi","aic")
moreAFIs <- c("gammaHat","adjGammaHat")
## Use only 20 permutations for a demo.  In practice,
## use > 1000 to reduce sampling variability of estimated p values
## test configural invariance
set.seed(12345)
out.config <- permuteMeasEq(nPermute = 20, con = fit.config)
out.config
## test metric equivalence
set.seed(12345) # same permutations
out.metric <- permuteMeasEq(nPermute = 20, uncon = fit.config, con = fit.metric,
                            param = "loadings", AFIs = myAFIs,
                            moreAFIs = moreAFIs, null = fit.null)
summary(out.metric, nd = 4)
""")

R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




  |==================================================| 100%
  |==================================================| 100%
Omnibus p value based on parametric chi-squared difference test:

Chisq diff    Df diff Pr(>Chisq) 
   14.0791    12.0000     0.2957 


Omnibus p values based on nonparametric permutation method: 

            AFI.Difference p.value
chisq              14.0791    0.50
cfi                -0.0022    0.50
rmsea              -0.0071    0.65
mfi                -0.0030    0.50
aic                -9.9209    0.50
gammaHat           -0.0013    0.50
adjGammaHat         0.0147    0.80


 Modification indices for equality constrained parameter estimates,
 with unadjusted 'p.value' based on chi-squared distribution and
 adjusted 'tukey.p.value' based on permutation distribution of the
 maximum modification index per iteration: 

    lhs op   rhs     X2 df p.value tukey.p.value flag   parameter group.lhs
1  .p1. == .p37. 1.8169  1  0.1777          0.95       visual=~x1  thirteen
2  

,,lhs,op,rhs,X2,...,group.lhs,group.rhs
0,1,.p1.,==,.p37.,1.8169024991429243,...,thirteen,preteen
1,2,.p1.,==,.p73.,1.4547462297192721,...,thirteen,teen
2,3,.p2.,==,.p38.,1.6106698869682952,...,thirteen,preteen
3,4,.p2.,==,.p74.,4.216329058513733,...,thirteen,teen
4,5,.p3.,==,.p39.,0.06966862102540441,...,thirteen,preteen
5,6,.p3.,==,.p75.,0.19363527176891665,...,thirteen,teen
6,7,.p4.,==,.p40.,0.02927082427311543,...,thirteen,preteen
7,8,.p4.,==,.p76.,1.1514857323896808,...,thirteen,teen
...,...,...,...,...,...,...,...,...
16,17,.p9.,==,.p45.,0.049187038826852346,...,thirteen,preteen


In [9]:
ro.r("""
library("lavaan")
library("semTools")
## create 3-group data in lavaan example(cfa) data
HS <- lavaan::HolzingerSwineford1939
head(HS,3)

""")

,,id,sex,ageyr,agemo,...,x8,x9
0,1,1,1,13,1,...,5.75,6.3611111
1,2,2,2,13,7,...,6.25,7.9166667
2,3,3,2,13,1,...,3.9,4.4166667


# ------------------------------------------------------------------------------------

In [10]:
# we need to import 
# -> HiTOP validation sample
# -> First assessment of genpop sample

## IMPORT HITOP VALIDATION SAMPLE


In [11]:
# this is a table that aligns scale items with their corresponding questions
htqs = pd.read_excel('../data/ValSample/Internalizing-Somatoform Items_DW.xlsx', skiprows=2,
                                names=['id', 'scale', 'item', '_0', '_1', '_2', '_3']).drop(['_0', '_1', '_2', '_3'], axis=1)
htqs.head(3)

/Users/zeleninam2/Documents/projects/hitop/2-analysis-CFA/env/lib/python3.9/site-packages/openpyxl/worksheet/_read_only.py:81: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


,id,scale,item
0,1,Social Anxiety,I felt shy around other people.
1,2,Health Anxiety,I kept imagining diseases I might have
2,3,Perfectionism,I demanded perfection from myself.


In [12]:
# cmqs is a dataframe with items and responses
# htcm is the same dataframe, but only displaying HiTOP items
cmqs = pd.read_csv('../data/CFA/cogmood_questions.csv')
cmqs = cmqs.rename({'Unnamed: 0': 'id'}, axis=1)
htcm = cmqs.query("measure == 'HiTOP'")
htcm.head(1)

,id,measure,page_number,page_title,subscale,Prompt,item,response 1,response 2,response 3,response 4,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27
42,3.0,HiTOP,7,NaN,anhedonic depression,Have there been significant times during the last two weeks during which the following statement...,I had very little energy.,Not at all,A little,Moderately,A lot,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
# this is a lookup table for subscales, items (sentence questions), and hitop ids
item_lookup = htcm.loc[:, ['id','subscale', 'item']].merge(htqs.loc[:, ['id','item']], how='inner', on='item', suffixes=['_cm', '_ht'])
item_lookup['htid'] = 'hitop'+ item_lookup.id_ht.astype(str)
item_lookup

,id_cm,subscale,item,id_ht,htid
0,3.0,anhedonic depression,I had very little energy.,157,hitop157
1,9.0,separation insecurity,I worried that others would abandon me.,81,hitop81
2,16.0,anxious worry,Thoughts were racing through my head.,34,hitop34
3,19.0,well-being,It was easy for me to laugh.,54,hitop54
4,23.0,appetite gain,I stuffed myself with food.,243,hitop243
5,27.0,anhedonic depression,I was unable to enjoy things like I normally do.,182,hitop182
6,28.0,separation insecurity,I wanted other people to take care of me.,69,hitop69
7,41.0,anxious worry,I had a lot of nervous energy.,89,hitop89
8,43.0,separation insecurity,I wanted someone else to make decisions for me.,50,hitop50
9,47.0,social anxiety,I avoided performing or giving a talk in front of others.,129,hitop129


In [14]:
item_lookup.loc[item_lookup['htid'] == 'hitop123']

,id_cm,subscale,item,id_ht,htid
62,301.0,anhedonic depression,Nothing made me laugh.,123,hitop123


In [15]:
# VALIDATION SAMPLE - main dataframe with responses
responses = pd.read_excel('../data/ValSample/inters23.xlsx')
resp_cols = responses.columns[responses.columns.isin(item_lookup.htid)]
responses.head(3)

,gender,race,hispanic,current_txt,past_txt,current_med,age,hitop39,hitop77,hitop84,hitop92,hitop93,hitop123,hitop157,hitop182,hitop230,hitop246,hitop20,hitop34,hitop89,hitop203,hitop240,hitop248,hitop265,hitop120,hitop141,hitop243,hitop275,hitop109,hitop280,hitop283,hitop67,hitop142,hitop159,hitop189,hitop5,hitop66,hitop99,hitop181,hitop231,hitop21,hitop90,hitop95,hitop160,hitop254,hitop261,hitop268,hitop15,hitop104,hitop126,hitop211,hitop215,hitop257,hitop40,hitop50,hitop69,hitop81,hitop113,hitop136,hitop151,hitop197,hitop72,hitop140,hitop143,hitop220,hitop16,hitop165,hitop225,hitop247,hitop278,hitop1,hitop17,hitop114,hitop117,hitop124,hitop129,hitop204,hitop222,hitop236,hitop258,hitop9,hitop23,hitop54,hitop106,hitop149,hitop200,hitop244,hitop245,hitop250,hitop281
0,1,5,2,1,1,1,24,4,3,3,3,4,1,3,2,2,2,3,3,2,2,2,1,2,1,2,2,2,2,2,3,3,2,2,2,2,3,2,1,1,3,2,2,1,2,3,1,3,1,3,2,1,2,3,2,2,1,3,2,2,2,3,2,2,2,1,2,1,4,1,3,4,2,3,2,1,3,1,2,1,3,3,3,2,2,2,2,2,1,2
1,1,2,2,2,1,2,32,2,1,1,2,2,1,2,2,2,1,2,3,2,2,1,1,2,1,1,2,1,1,1,1,2,2,1,2,2,1,1,1,1,2,1,1,2,3,2,2,2,1,1,1,1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,2,1,3,1,1,1,1,1,1,1,1,1,1,1,2,3,3,3,3,2,1,3,3,3
2,2,5,2,2,2,1,68,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,1,1,1,1,1,1,2,1,1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,4,1,1,1,1,2,1,1,1,1,1,1,3,4,4,4,4,3,4,4,4,4


In [16]:
resp_cols

Index(['hitop39', 'hitop77', 'hitop84', 'hitop92', 'hitop93', 'hitop123',
       'hitop157', 'hitop182', 'hitop230', 'hitop246', 'hitop20', 'hitop34',
       'hitop89', 'hitop203', 'hitop248', 'hitop265', 'hitop120', 'hitop141',
       'hitop243', 'hitop275', 'hitop109', 'hitop280', 'hitop283', 'hitop142',
       'hitop159', 'hitop189', 'hitop5', 'hitop66', 'hitop99', 'hitop181',
       'hitop231', 'hitop21', 'hitop90', 'hitop95', 'hitop160', 'hitop254',
       'hitop261', 'hitop268', 'hitop15', 'hitop104', 'hitop126', 'hitop211',
       'hitop215', 'hitop257', 'hitop40', 'hitop50', 'hitop69', 'hitop81',
       'hitop113', 'hitop136', 'hitop151', 'hitop197', 'hitop72', 'hitop140',
       'hitop143', 'hitop220', 'hitop16', 'hitop165', 'hitop225', 'hitop247',
       'hitop278', 'hitop1', 'hitop17', 'hitop114', 'hitop117', 'hitop124',
       'hitop129', 'hitop204', 'hitop222', 'hitop236', 'hitop258', 'hitop9',
       'hitop23', 'hitop54', 'hitop106', 'hitop149', 'hitop200', 'hitop244',
  

In [17]:
scale_scores = []
for ss, df in item_lookup.groupby('subscale'):
    print(ss)
    print(df.sort_values('id_ht').htid.values)
    tmp = responses.loc[:, df.htid].sum(1)
    tmp.name = ss.replace(" ", "_").replace("/", "_").replace("-", "_")
    scale_scores.append(tmp)
scale_scores = pd.concat(scale_scores, axis=1)

anhedonic depression
['hitop39' 'hitop77' 'hitop84' 'hitop92' 'hitop93' 'hitop123' 'hitop157'
 'hitop182' 'hitop230' 'hitop246']
anxious worry
['hitop20' 'hitop34' 'hitop89' 'hitop203' 'hitop248' 'hitop265']
appetite gain
['hitop120' 'hitop141' 'hitop243' 'hitop275']
appetite loss
['hitop109' 'hitop280' 'hitop283']
cognitive problems
['hitop142' 'hitop159' 'hitop189']
hyposomnia
['hitop5' 'hitop66' 'hitop99' 'hitop181' 'hitop231']
indecisiveness
['hitop21' 'hitop90' 'hitop95']
insomnia
['hitop160' 'hitop254' 'hitop261' 'hitop268']
panic
['hitop15' 'hitop104' 'hitop126' 'hitop211' 'hitop215' 'hitop257']
separation insecurity
['hitop40' 'hitop50' 'hitop69' 'hitop81' 'hitop113' 'hitop136' 'hitop151'
 'hitop197']
shame/guilt
['hitop72' 'hitop140' 'hitop143' 'hitop220']
situational phobia
['hitop16' 'hitop165' 'hitop225' 'hitop247' 'hitop278']
social anxiety
['hitop1' 'hitop17' 'hitop114' 'hitop117' 'hitop124' 'hitop129' 'hitop204'
 'hitop222' 'hitop236' 'hitop258']
well-being
['hitop9' 'hi

In [18]:
scale_names = list(scale_scores.columns.values)
scale_names

['anhedonic_depression',
 'anxious_worry',
 'appetite_gain',
 'appetite_loss',
 'cognitive_problems',
 'hyposomnia',
 'indecisiveness',
 'insomnia',
 'panic',
 'separation_insecurity',
 'shame_guilt',
 'situational_phobia',
 'social_anxiety',
 'well_being']

In [19]:
# this is the validation data to be used
dat_validation = responses
dat_validation.head(3)

,gender,race,hispanic,current_txt,past_txt,current_med,age,hitop39,hitop77,hitop84,hitop92,hitop93,hitop123,hitop157,hitop182,hitop230,hitop246,hitop20,hitop34,hitop89,hitop203,hitop240,hitop248,hitop265,hitop120,hitop141,hitop243,hitop275,hitop109,hitop280,hitop283,hitop67,hitop142,hitop159,hitop189,hitop5,hitop66,hitop99,hitop181,hitop231,hitop21,hitop90,hitop95,hitop160,hitop254,hitop261,hitop268,hitop15,hitop104,hitop126,hitop211,hitop215,hitop257,hitop40,hitop50,hitop69,hitop81,hitop113,hitop136,hitop151,hitop197,hitop72,hitop140,hitop143,hitop220,hitop16,hitop165,hitop225,hitop247,hitop278,hitop1,hitop17,hitop114,hitop117,hitop124,hitop129,hitop204,hitop222,hitop236,hitop258,hitop9,hitop23,hitop54,hitop106,hitop149,hitop200,hitop244,hitop245,hitop250,hitop281
0,1,5,2,1,1,1,24,4,3,3,3,4,1,3,2,2,2,3,3,2,2,2,1,2,1,2,2,2,2,2,3,3,2,2,2,2,3,2,1,1,3,2,2,1,2,3,1,3,1,3,2,1,2,3,2,2,1,3,2,2,2,3,2,2,2,1,2,1,4,1,3,4,2,3,2,1,3,1,2,1,3,3,3,2,2,2,2,2,1,2
1,1,2,2,2,1,2,32,2,1,1,2,2,1,2,2,2,1,2,3,2,2,1,1,2,1,1,2,1,1,1,1,2,2,1,2,2,1,1,1,1,2,1,1,2,3,2,2,2,1,1,1,1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,2,1,3,1,1,1,1,1,1,1,1,1,1,1,2,3,3,3,3,2,1,3,3,3
2,2,5,2,2,2,1,68,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,1,1,1,1,1,1,2,1,1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,4,1,1,1,1,2,1,1,1,1,1,1,3,4,4,4,4,3,4,4,4,4


## IMPORT HITOP GENPOP

In [20]:
# we need to preprocess it differently - looking at the notebook_1_preprocessing but editing the code from there as needed

In [21]:
mydat_genpop_raw = pd.read_csv('../data/NIMH0007_genpop_num_OUTPUT.csv')
mydat_genpop_raw.head()

,caseid,weight,sample_type,screener_1,screener_2,screener_3_1,screener_3_2,screener_3_3,consent,FNM_Q24,FNM_Q25_1,FNM_Q25_2,FNM_Q25_3,FNM_Q25_4,FNM_Q25_5,FNM_Q25_6,FNM_Q25_955,FNM_Q25_933,FNM_Q25_open1,FNM_Q26_d,FNM_Q27_1,FNM_Q27_2,FNM_Q27_3,FNM_Q27_4,FNM_Q27_5,FNM_Q27_6,FNM_Q27_7,FNM_Q27_8,FNM_Q27_9,FNM_Q27_10,FNM_Q27_11,FNM_Q27_12,FNM_Q27_13,FNM_Q27_14,FNM_Q27_15,FNM_Q27_16,FNM_Q27_17,FNM_Q27_18,FNM_Q27_19,FNM_Q27_955,FNM_Q27_933,FNM_Q27_open1,FNM_Q28,FNM_Q29,FNM_Q30_m_1,FNM_Q30_m_2,FNM_Q30_m_3,FNM_Q30_m_4,FNM_Q30_m_5,FNM_Q30_m_6,FNM_Q30_m_7,FNM_Q30_m_8,FNM_Q30_m_955,FNM_Q30_m_933,FNM_Q30_open1,FNM_Q31_d,FNM_Q32_1,FNM_Q32_2,FNM_Q32_3,FNM_Q32_4,FNM_Q32_5,FNM_Q32_6,FNM_Q32_7,FNM_Q32_8,FNM_Q32_9,FNM_Q32_10,FNM_Q32_955,FNM_Q32_933,FNM_Q32_open1,FNM_Q33,FNM_34,FNM_Q35_m_1,FNM_Q35_m_2,FNM_Q35_m_3,FNM_Q35_m_933,FNM_Q35_open1,FNM_Q36_d,FNM_Q37_m_1,FNM_Q37_m_2,FNM_Q37_m_3,FNM_Q37_m_4,FNM_Q37_m_5,FNM_Q37_m_6,FNM_Q37_m_7,FNM_Q37_m_8,FNM_Q37_m_9,FNM_Q37_m_955,FNM_Q37_m_933,FNM_Q37_open1,FNM_Q38,FNM_Q1_1,FNM_Q1_2,FNM_Q1_3,FNM_Q1_4,FNM_Q1_5,FNM_Q1_6,FNM_Q1_attn,FNM_Q1_7,FNM_Q1_8,FNM_Q1_9,FNM_Q2_1,FNM_Q2_2,FNM_Q2_3,FNM_Q2_4,FNM_Q2_5,FNM_Q3_1,FNM_Q3_2,FNM_Q3_3,FNM_Q3_4,FNM_Q4_1,FNM_Q4_2,FNM_Q4_3,FNM_Q4_4,FNM_Q4_5,FNM_Q4_6,FNM_Q4_7,FNM_Q4_8,FNM_Q4_9,FNM_Q5_1,FNM_Q5_2,FNM_Q5_3,FNM_Q5_4,FNM_Q5_5,FNM_Q5_6,FNM_Q5_7,FNM_Q6_1,FNM_Q6_2,FNM_Q6_3,FNM_Q6_4,FNM_Q6_5,FNM_Q6_6,FNM_Q6_7,FNM_Q6_8,FNM_Q7_1,FNM_Q7_2,FNM_Q7_3,FNM_Q7_4,FNM_Q7_5,FNM_Q8_1,FNM_Q8_2,FNM_Q8_3,FNM_Q8_4,FNM_Q8_5,FNM_Q8_6,FNM_Q9_1,FNM_Q9_2,FNM_Q9_3,FNM_Q9_4,FNM_Q9_5,FNM_Q10_1,FNM_Q10_2,FNM_Q10_3,FNM_Q10_4,FNM_Q10_5,FNM_Q11_1,FNM_Q11_2,FNM_Q11_3,FNM_Q11_4,FNM_Q11_5,FNM_Q12_1,FNM_Q12_2,FNM_Q12_3,FNM_Q12_4,FNM_Q12_5,FNM_Q13_1,FNM_Q13_2,FNM_Q13_3,FNM_Q13_4,FNM_Q13_5,FNM_Q14_1,FNM_Q14_2,FNM_Q14_3,FNM_Q14_4,FNM_Q14_5,FNM_Q15_1,FNM_Q15_2,FNM_Q15_3,FNM_Q15_4,FNM_Q15_5,FNM_Q16_1,FNM_Q16_2,FNM_Q16_3,FNM_Q16_4,FNM_Q16_5,FNM_Q17_1,FNM_Q17_2,FNM_Q17_3,FNM_Q17_4,FNM_Q17_5,FNM_Q18_1,FNM_Q18_2,FNM_Q18_3,FNM_Q18_4,FNM_Q18_5,FNM_Q19_1,FNM_Q19_2,FNM_Q19_3,FNM_Q19_4,FNM_Q19_5,FNM_Q20_1,FNM_Q20_2,FNM_Q20_3,FNM_Q20_4,FNM_Q20_5,FNM_Q21_1,FNM_Q21_2,FNM_Q21_3,FNM_Q21_4,FNM_Q21_5,FNM_Q22_1,FNM_Q22_2,FNM_Q22_3,FNM_Q22_4,FNM_Q22_5,FNM_Q22_6,FNM_Q23_1,FNM_Q23_2,FNM_Q23_3,FNM_Q40_m_1,FNM_Q40_m_2,FNM_Q40_m_3,FNM_Q40_m_4,FNM_Q40_m_5,FNM_Q40_m_6,FNM_Q40_m_7,FNM_Q40_m_8,FNM_Q40_m_9,FNM_Q40_m_10,FNM_Q40_m_11,FNM_Q40_m_12,FNM_Q40_m_13,FNM_Q40_m_14,FNM_Q40_m_15,FNM_Q40_m_16,FNM_Q40_m_17,FNM_Q40_m_18,FNM_Q40_m_19,FNM_Q40_m_20,FNM_Q40_m_21,FNM_Q40_m_22,FNM_Q40_m_23,FNM_Q40_m_24,FNM_Q40_m_25,FNM_Q40_m_26,FNM_Q40_m_27,FNM_Q40_m_944,FNM_Q41_m_1,FNM_Q41_m_2,FNM_Q41_m_3,FNM_Q41_m_4,...,FNM_Q3_1_recontact,FNM_Q3_2_recontact,FNM_Q3_3_recontact,FNM_Q3_4_recontact,FNM_Q4_1_recontact,FNM_Q4_2_recontact,FNM_Q4_3_recontact,FNM_Q4_4_recontact,FNM_Q4_5_recontact,FNM_Q4_6_recontact,FNM_Q4_7_recontact,FNM_Q4_8_recontact,FNM_Q4_9_recontact,FNM_Q5_1_recontact,FNM_Q5_2_recontact,FNM_Q5_3_recontact,FNM_Q5_4_recontact,FNM_Q5_5_recontact,FNM_Q5_6_recontact,FNM_Q5_7_recontact,FNM_Q6_1_recontact,FNM_Q6_2_recontact,FNM_Q6_3_recontact,FNM_Q6_4_recontact,FNM_Q6_5_recontact,FNM_Q6_6_recontact,FNM_Q6_7_recontact,FNM_Q6_8_recontact,FNM_Q7_1_recontact,FNM_Q7_2_recontact,FNM_Q7_3_recontact,FNM_Q7_4_recontact,FNM_Q7_5_recontact,FNM_Q8_1_recontact,FNM_Q8_2_recontact,FNM_Q8_3_recontact,FNM_Q8_4_recontact,FNM_Q8_5_recontact,FNM_Q8_6_recontact,FNM_Q9_1_recontact,FNM_Q9_2_recontact,FNM_Q9_3_recontact,FNM_Q9_4_recontact,FNM_Q9_5_recontact,FNM_Q10_1_recontact,FNM_Q10_2_recontact,FNM_Q10_3_recontact,FNM_Q10_4_recontact,FNM_Q10_5_recontact,FNM_Q11_1_recontact,FNM_Q11_2_recontact,FNM_Q11_3_recontact,FNM_Q11_4_recontact,FNM_Q11_5_recontact,FNM_Q12_1_recontact,FNM_Q12_2_recontact,FNM_Q12_3_recontact,FNM_Q12_4_recontact,FNM_Q12_5_recontact,FNM_Q13_1_recontact,FNM_Q13_2_recontact,FNM_Q13_3_recontact,FNM_Q13_4_recontact,FNM_Q13_5_recontact,FNM_Q14_1_recontact,FNM_Q14_2_recontact,FNM_Q14_3_recontact,FNM_Q14_4_recontact,FNM_Q14_5_recontact,FNM_Q15_1_recontact,FNM_Q15_2_re

In [22]:
#loads the codesheet file
# wherever the emane column exists, creates a dict variablename:ename
code_book = pd.read_csv('../dylan_github/yougov_codesheet_alignment.tsv', sep='\t')
ename_lut = {vn:en for vn, en in code_book.loc[code_book.ename.notnull(), ['var_name', 'ename']].values}
ename_lut

{'caseid': 'caseid',
 'birthyr': 'birthyr',
 'gender': 'gender',
 'sex': 'sex',
 'educ': 'educ',
 'inputstate': 'inputstate',
 'sample_type': 'sample_type',
 'screener_1': 'screener_1ongoing',
 'screener_2': 'screener_2impact',
 'screener_3_1': 'screener_3depression',
 'screener_3_2': 'screener_4anxiety',
 'screener_3_3': 'screener_5attention',
 'consent': 'consent',
 'FNM_Q24': 'mood_yn',
 'FNM_Q26_d': 'mood_years',
 'FNM_Q28': 'mood_bothered',
 'FNM_Q29': 'anxiety_yn',
 'FNM_Q31_d': 'anxiety_years',
 'FNM_Q33': 'anxiety_bothered',
 'FNM_34': 'attention_yn',
 'FNM_Q36_d': 'attention_years',
 'FNM_Q38': 'attention_bothered',
 'FNM_Q1_1': 'inattention_1',
 'FNM_Q1_2': 'inattention_2',
 'FNM_Q1_3': 'inattention_3',
 'FNM_Q1_4': 'inattention_4',
 'FNM_Q1_5': 'inattention_5',
 'FNM_Q1_6': 'inattention_6',
 'FNM_Q1_7': 'inattention_7',
 'FNM_Q1_8': 'inattention_8',
 'FNM_Q1_9': 'inattention_9',
 'FNM_Q2_1': 'hyperactivity_1',
 'FNM_Q2_2': 'hyperactivity_2',
 'FNM_Q2_3': 'hyperactivity_3',
 

In [23]:
for vn, en in code_book.loc[code_book.ename.notnull(), ['var_name', 'ename']].values:
    ename_lut[vn+'_recontact'] = en+'_recontact'
ename_lut

{'caseid': 'caseid',
 'birthyr': 'birthyr',
 'gender': 'gender',
 'sex': 'sex',
 'educ': 'educ',
 'inputstate': 'inputstate',
 'sample_type': 'sample_type',
 'screener_1': 'screener_1ongoing',
 'screener_2': 'screener_2impact',
 'screener_3_1': 'screener_3depression',
 'screener_3_2': 'screener_4anxiety',
 'screener_3_3': 'screener_5attention',
 'consent': 'consent',
 'FNM_Q24': 'mood_yn',
 'FNM_Q26_d': 'mood_years',
 'FNM_Q28': 'mood_bothered',
 'FNM_Q29': 'anxiety_yn',
 'FNM_Q31_d': 'anxiety_years',
 'FNM_Q33': 'anxiety_bothered',
 'FNM_34': 'attention_yn',
 'FNM_Q36_d': 'attention_years',
 'FNM_Q38': 'attention_bothered',
 'FNM_Q1_1': 'inattention_1',
 'FNM_Q1_2': 'inattention_2',
 'FNM_Q1_3': 'inattention_3',
 'FNM_Q1_4': 'inattention_4',
 'FNM_Q1_5': 'inattention_5',
 'FNM_Q1_6': 'inattention_6',
 'FNM_Q1_7': 'inattention_7',
 'FNM_Q1_8': 'inattention_8',
 'FNM_Q1_9': 'inattention_9',
 'FNM_Q2_1': 'hyperactivity_1',
 'FNM_Q2_2': 'hyperactivity_2',
 'FNM_Q2_3': 'hyperactivity_3',
 

In [24]:
# this groups hitop items by subscales
# marie: add the initial and the recontact scales

tmp = code_book.query('ename.notnull()')
scale_lut = {}
for ss, df in tmp.loc[tmp.ename.str.contains('hitop') & ~tmp.ename.str.contains('today')].groupby('subscale'):
    scale_name = ss.replace(" ", "_").replace("/", "_").replace("-", "_")
    items = df.ename.values
    scale_lut[scale_name] = items
    
 #   scale_name_initial = scale_name+'_initial'
 #   scale_lut[scale_name_initial] = items 
    scale_name_recontact = scale_name+'_recontact'
    items_recontact = []
    for item in items:
        items_recontact.append(item+'_recontact')
    scale_lut[scale_name_recontact] = np.array(items_recontact, dtype=object)
scale_lut

{'anhedonic_depression': array(['hitop157', 'hitop182', 'hitop93', 'hitop84', 'hitop77', 'hitop92',
        'hitop39', 'hitop246', 'hitop230', 'hitop123'], dtype=object),
 'anhedonic_depression_recontact': array(['hitop157_recontact', 'hitop182_recontact', 'hitop93_recontact',
        'hitop84_recontact', 'hitop77_recontact', 'hitop92_recontact',
        'hitop39_recontact', 'hitop246_recontact', 'hitop230_recontact',
        'hitop123_recontact'], dtype=object),
 'anxious_worry': array(['hitop34', 'hitop89', 'hitop265', 'hitop20', 'hitop248',
        'hitop240', 'hitop203'], dtype=object),
 'anxious_worry_recontact': array(['hitop34_recontact', 'hitop89_recontact', 'hitop265_recontact',
        'hitop20_recontact', 'hitop248_recontact', 'hitop240_recontact',
        'hitop203_recontact'], dtype=object),
 'appetite_gain': array(['hitop243', 'hitop141', 'hitop120', 'hitop275'], dtype=object),
 'appetite_gain_recontact': array(['hitop243_recontact', 'hitop141_recontact', 'hitop120_recont

In [25]:
# done working with the codesheet file, now opening the actual data

In [26]:
datapath = '../data/NIMH0007_genpop_num_OUTPUT.csv'
dat = pd.read_csv(datapath, dtype={'caseid':str}, engine='python')
dat.head()

,caseid,weight,sample_type,screener_1,screener_2,screener_3_1,screener_3_2,screener_3_3,consent,FNM_Q24,FNM_Q25_1,FNM_Q25_2,FNM_Q25_3,FNM_Q25_4,FNM_Q25_5,FNM_Q25_6,FNM_Q25_955,FNM_Q25_933,FNM_Q25_open1,FNM_Q26_d,FNM_Q27_1,FNM_Q27_2,FNM_Q27_3,FNM_Q27_4,FNM_Q27_5,FNM_Q27_6,FNM_Q27_7,FNM_Q27_8,FNM_Q27_9,FNM_Q27_10,FNM_Q27_11,FNM_Q27_12,FNM_Q27_13,FNM_Q27_14,FNM_Q27_15,FNM_Q27_16,FNM_Q27_17,FNM_Q27_18,FNM_Q27_19,FNM_Q27_955,FNM_Q27_933,FNM_Q27_open1,FNM_Q28,FNM_Q29,FNM_Q30_m_1,FNM_Q30_m_2,FNM_Q30_m_3,FNM_Q30_m_4,FNM_Q30_m_5,FNM_Q30_m_6,FNM_Q30_m_7,FNM_Q30_m_8,FNM_Q30_m_955,FNM_Q30_m_933,FNM_Q30_open1,FNM_Q31_d,FNM_Q32_1,FNM_Q32_2,FNM_Q32_3,FNM_Q32_4,FNM_Q32_5,FNM_Q32_6,FNM_Q32_7,FNM_Q32_8,FNM_Q32_9,FNM_Q32_10,FNM_Q32_955,FNM_Q32_933,FNM_Q32_open1,FNM_Q33,FNM_34,FNM_Q35_m_1,FNM_Q35_m_2,FNM_Q35_m_3,FNM_Q35_m_933,FNM_Q35_open1,FNM_Q36_d,FNM_Q37_m_1,FNM_Q37_m_2,FNM_Q37_m_3,FNM_Q37_m_4,FNM_Q37_m_5,FNM_Q37_m_6,FNM_Q37_m_7,FNM_Q37_m_8,FNM_Q37_m_9,FNM_Q37_m_955,FNM_Q37_m_933,FNM_Q37_open1,FNM_Q38,FNM_Q1_1,FNM_Q1_2,FNM_Q1_3,FNM_Q1_4,FNM_Q1_5,FNM_Q1_6,FNM_Q1_attn,FNM_Q1_7,FNM_Q1_8,FNM_Q1_9,FNM_Q2_1,FNM_Q2_2,FNM_Q2_3,FNM_Q2_4,FNM_Q2_5,FNM_Q3_1,FNM_Q3_2,FNM_Q3_3,FNM_Q3_4,FNM_Q4_1,FNM_Q4_2,FNM_Q4_3,FNM_Q4_4,FNM_Q4_5,FNM_Q4_6,FNM_Q4_7,FNM_Q4_8,FNM_Q4_9,FNM_Q5_1,FNM_Q5_2,FNM_Q5_3,FNM_Q5_4,FNM_Q5_5,FNM_Q5_6,FNM_Q5_7,FNM_Q6_1,FNM_Q6_2,FNM_Q6_3,FNM_Q6_4,FNM_Q6_5,FNM_Q6_6,FNM_Q6_7,FNM_Q6_8,FNM_Q7_1,FNM_Q7_2,FNM_Q7_3,FNM_Q7_4,FNM_Q7_5,FNM_Q8_1,FNM_Q8_2,FNM_Q8_3,FNM_Q8_4,FNM_Q8_5,FNM_Q8_6,FNM_Q9_1,FNM_Q9_2,FNM_Q9_3,FNM_Q9_4,FNM_Q9_5,FNM_Q10_1,FNM_Q10_2,FNM_Q10_3,FNM_Q10_4,FNM_Q10_5,FNM_Q11_1,FNM_Q11_2,FNM_Q11_3,FNM_Q11_4,FNM_Q11_5,FNM_Q12_1,FNM_Q12_2,FNM_Q12_3,FNM_Q12_4,FNM_Q12_5,FNM_Q13_1,FNM_Q13_2,FNM_Q13_3,FNM_Q13_4,FNM_Q13_5,FNM_Q14_1,FNM_Q14_2,FNM_Q14_3,FNM_Q14_4,FNM_Q14_5,FNM_Q15_1,FNM_Q15_2,FNM_Q15_3,FNM_Q15_4,FNM_Q15_5,FNM_Q16_1,FNM_Q16_2,FNM_Q16_3,FNM_Q16_4,FNM_Q16_5,FNM_Q17_1,FNM_Q17_2,FNM_Q17_3,FNM_Q17_4,FNM_Q17_5,FNM_Q18_1,FNM_Q18_2,FNM_Q18_3,FNM_Q18_4,FNM_Q18_5,FNM_Q19_1,FNM_Q19_2,FNM_Q19_3,FNM_Q19_4,FNM_Q19_5,FNM_Q20_1,FNM_Q20_2,FNM_Q20_3,FNM_Q20_4,FNM_Q20_5,FNM_Q21_1,FNM_Q21_2,FNM_Q21_3,FNM_Q21_4,FNM_Q21_5,FNM_Q22_1,FNM_Q22_2,FNM_Q22_3,FNM_Q22_4,FNM_Q22_5,FNM_Q22_6,FNM_Q23_1,FNM_Q23_2,FNM_Q23_3,FNM_Q40_m_1,FNM_Q40_m_2,FNM_Q40_m_3,FNM_Q40_m_4,FNM_Q40_m_5,FNM_Q40_m_6,FNM_Q40_m_7,FNM_Q40_m_8,FNM_Q40_m_9,FNM_Q40_m_10,FNM_Q40_m_11,FNM_Q40_m_12,FNM_Q40_m_13,FNM_Q40_m_14,FNM_Q40_m_15,FNM_Q40_m_16,FNM_Q40_m_17,FNM_Q40_m_18,FNM_Q40_m_19,FNM_Q40_m_20,FNM_Q40_m_21,FNM_Q40_m_22,FNM_Q40_m_23,FNM_Q40_m_24,FNM_Q40_m_25,FNM_Q40_m_26,FNM_Q40_m_27,FNM_Q40_m_944,FNM_Q41_m_1,FNM_Q41_m_2,FNM_Q41_m_3,FNM_Q41_m_4,...,FNM_Q3_1_recontact,FNM_Q3_2_recontact,FNM_Q3_3_recontact,FNM_Q3_4_recontact,FNM_Q4_1_recontact,FNM_Q4_2_recontact,FNM_Q4_3_recontact,FNM_Q4_4_recontact,FNM_Q4_5_recontact,FNM_Q4_6_recontact,FNM_Q4_7_recontact,FNM_Q4_8_recontact,FNM_Q4_9_recontact,FNM_Q5_1_recontact,FNM_Q5_2_recontact,FNM_Q5_3_recontact,FNM_Q5_4_recontact,FNM_Q5_5_recontact,FNM_Q5_6_recontact,FNM_Q5_7_recontact,FNM_Q6_1_recontact,FNM_Q6_2_recontact,FNM_Q6_3_recontact,FNM_Q6_4_recontact,FNM_Q6_5_recontact,FNM_Q6_6_recontact,FNM_Q6_7_recontact,FNM_Q6_8_recontact,FNM_Q7_1_recontact,FNM_Q7_2_recontact,FNM_Q7_3_recontact,FNM_Q7_4_recontact,FNM_Q7_5_recontact,FNM_Q8_1_recontact,FNM_Q8_2_recontact,FNM_Q8_3_recontact,FNM_Q8_4_recontact,FNM_Q8_5_recontact,FNM_Q8_6_recontact,FNM_Q9_1_recontact,FNM_Q9_2_recontact,FNM_Q9_3_recontact,FNM_Q9_4_recontact,FNM_Q9_5_recontact,FNM_Q10_1_recontact,FNM_Q10_2_recontact,FNM_Q10_3_recontact,FNM_Q10_4_recontact,FNM_Q10_5_recontact,FNM_Q11_1_recontact,FNM_Q11_2_recontact,FNM_Q11_3_recontact,FNM_Q11_4_recontact,FNM_Q11_5_recontact,FNM_Q12_1_recontact,FNM_Q12_2_recontact,FNM_Q12_3_recontact,FNM_Q12_4_recontact,FNM_Q12_5_recontact,FNM_Q13_1_recontact,FNM_Q13_2_recontact,FNM_Q13_3_recontact,FNM_Q13_4_recontact,FNM_Q13_5_recontact,FNM_Q14_1_recontact,FNM_Q14_2_recontact,FNM_Q14_3_recontact,FNM_Q14_4_recontact,FNM_Q14_5_recontact,FNM_Q15_1_recontact,FNM_Q15_2_re

In [27]:
dat.columns

Index(['caseid', 'weight', 'sample_type', 'screener_1', 'screener_2',
       'screener_3_1', 'screener_3_2', 'screener_3_3', 'consent', 'FNM_Q24',
       ...
       'FNM_44_m_31_recontact', 'FNM_44_m_32_recontact',
       'FNM_44_m_33_recontact', 'FNM_44_m_34_recontact',
       'FNM_44_m_35_recontact', 'family_wealth_recontact',
       'WHODAS_H2_recontact', 'WHODAS_H3_recontact', 'starttime_recontact',
       'endtime_recontact'],
      dtype='object', length=728)

In [28]:
# drop the .0 that pandas appends for some reason
dat['caseid'] = dat.caseid.str[:-2]
# i'm not sure what this does
dat = dat.rename(ename_lut, axis=1)

In [29]:
dat_genpop = dat
dat_genpop.head(3)

,caseid,weight,sample_type,screener_1ongoing,screener_2impact,screener_3depression,screener_4anxiety,screener_5attention,consent,mood_yn,FNM_Q25_1,FNM_Q25_2,FNM_Q25_3,FNM_Q25_4,FNM_Q25_5,FNM_Q25_6,FNM_Q25_955,FNM_Q25_933,FNM_Q25_open1,mood_years,FNM_Q27_1,FNM_Q27_2,FNM_Q27_3,FNM_Q27_4,FNM_Q27_5,FNM_Q27_6,FNM_Q27_7,FNM_Q27_8,FNM_Q27_9,FNM_Q27_10,FNM_Q27_11,FNM_Q27_12,FNM_Q27_13,FNM_Q27_14,FNM_Q27_15,FNM_Q27_16,FNM_Q27_17,FNM_Q27_18,FNM_Q27_19,FNM_Q27_955,FNM_Q27_933,FNM_Q27_open1,mood_bothered,anxiety_yn,FNM_Q30_m_1,FNM_Q30_m_2,FNM_Q30_m_3,FNM_Q30_m_4,FNM_Q30_m_5,FNM_Q30_m_6,FNM_Q30_m_7,FNM_Q30_m_8,FNM_Q30_m_955,FNM_Q30_m_933,FNM_Q30_open1,anxiety_years,FNM_Q32_1,FNM_Q32_2,FNM_Q32_3,FNM_Q32_4,FNM_Q32_5,FNM_Q32_6,FNM_Q32_7,FNM_Q32_8,FNM_Q32_9,FNM_Q32_10,FNM_Q32_955,FNM_Q32_933,FNM_Q32_open1,anxiety_bothered,attention_yn,FNM_Q35_m_1,FNM_Q35_m_2,FNM_Q35_m_3,FNM_Q35_m_933,FNM_Q35_open1,attention_years,FNM_Q37_m_1,FNM_Q37_m_2,FNM_Q37_m_3,FNM_Q37_m_4,FNM_Q37_m_5,FNM_Q37_m_6,FNM_Q37_m_7,FNM_Q37_m_8,FNM_Q37_m_9,FNM_Q37_m_955,FNM_Q37_m_933,FNM_Q37_open1,attention_bothered,inattention_1,inattention_2,inattention_3,inattention_4,inattention_5,inattention_6,FNM_Q1_attn,inattention_7,inattention_8,inattention_9,hyperactivity_1,hyperactivity_2,hyperactivity_3,hyperactivity_4,hyperactivity_5,impulsivity_1,impulsivity_2,impulsivity_3,impulsivity_4,sct_1,sct_2,sct_3,sct_4,sct_5,sct_6,sct_7,sct_8,sct_9,gad_1,gad_2,gad_3,gad_4,gad_5,gad_6,gad_7,phq_1,phq_2,phq_3,phq_4,phq_5,phq_6,phq_7,phq_8,hitop157,hitop81,hitop34,hitop54,hitop243,hitop182,hitop69,hitop89,hitop50,check_moderately,hitop129,hitop265,hitop124,hitop231,hitop93,hitop67,hitop245,hitop281,hitop141,hitop40,hitop204,hitop21,hitop236,hitop280,hitop84,hitop120,hitop77,hitop92,hitop258,hitop39,hitop254,hitop215,hitop95,hitop106,hitop283,hitop16,hitop20,hitop189,hitop1,hitop136,hitop246,hitop248,hitop257,hitop114,hitop117,hitop250,hitop200,hitop160,hitop23,hitop165,hitop244,hitop9,hitop142,hitop230,hitop149,hitop247,hitop99,hitop66,hitop240,hitop222,hitop90,hitop113,hitop278,hitop203,hitop159,hitop123,hitop275,hitop268,hitop225,hitop143,hitop151,hitop181,hitop211,hitop17,hitop126,hitop5,hitop261,hitop220,check_notatall,hitop15,hitop72,hitop140,hitop109,hitop197,hitop104,todayinattention_1,todayinattention_2,todayinattention_3,todayinattention_4,todayinattention_5,todayinattention_6,todayinattention_7,todayinattention_8,todayinattention_9,todayhyperactivity_1,todayhyperactivity_2,todayhyperactivity_3,todayhyperactivity_4,todayhyperactivity_5,todayimpulsivity_1,todayimpulsivity_2,todayimpulsivity_3,todayimpulsivity_4,todaysct_1,todaysct_2,todaysct_3,todaysct_4,todaysct_5,todaysct_6,todaysct_7,todaysct_8,todaysct_9,today_na1,todaygad_1,todaygad_2,todaygad_3,todaygad_4,...,impulsivity_1_recontact,impulsivity_2_recontact,impulsivity_3_recontact,impulsivity_4_recontact,sct_1_recontact,sct_2_recontact,sct_3_recontact,sct_4_recontact,sct_5_recontact,sct_6_recontact,sct_7_recontact,sct_8_recontact,sct_9_recontact,gad_1_recontact,gad_2_recontact,gad_3_recontact,gad_4_recontact,gad_5_recontact,gad_6_recontact,gad_7_recontact,phq_1_recontact,phq_2_recontact,phq_3_recontact,phq_4_recontact,phq_5_recontact,phq_6_recontact,phq_7_recontact,phq_8_recontact,hitop157_recontact,hitop81_recontact,hitop34_recontact,hitop54_recontact,hitop243_recontact,hitop182_recontact,hitop69_recontact,hitop89_recontact,hitop50_recontact,check_moderately_recontact,hitop129_recontact,hitop265_recontact,hitop124_recontact,hitop231_recontact,hitop93_recontact,hitop67_recontact,hitop245_recontact,hitop281_recontact,hitop141_recontact,hitop40_recontact,hitop204_recontact,hitop21_recontact,hitop236_recontact,hitop280_recontact,hitop84_recontact,hitop120_recontact,hitop77_recontact,hitop92_recontact,hitop258_recontact,hitop39_recontact,hitop254_recontact,hitop215_recontact,hitop95_recontact,hitop106_recontact,hitop283_recontact,hitop16_recontact,hitop20_recontact,hitop189_recontact,hitop1_recontact,hitop136_recontact,hitop246_reconta

## check how dat dataframes look like

In [30]:
dat_validation.head()

,gender,race,hispanic,current_txt,past_txt,current_med,age,hitop39,hitop77,hitop84,hitop92,hitop93,hitop123,hitop157,hitop182,hitop230,hitop246,hitop20,hitop34,hitop89,hitop203,hitop240,hitop248,hitop265,hitop120,hitop141,hitop243,hitop275,hitop109,hitop280,hitop283,hitop67,hitop142,hitop159,hitop189,hitop5,hitop66,hitop99,hitop181,hitop231,hitop21,hitop90,hitop95,hitop160,hitop254,hitop261,hitop268,hitop15,hitop104,hitop126,hitop211,hitop215,hitop257,hitop40,hitop50,hitop69,hitop81,hitop113,hitop136,hitop151,hitop197,hitop72,hitop140,hitop143,hitop220,hitop16,hitop165,hitop225,hitop247,hitop278,hitop1,hitop17,hitop114,hitop117,hitop124,hitop129,hitop204,hitop222,hitop236,hitop258,hitop9,hitop23,hitop54,hitop106,hitop149,hitop200,hitop244,hitop245,hitop250,hitop281
0,1,5,2,1,1,1,24,4,3,3,3,4,1,3,2,2,2,3,3,2,2,2,1,2,1,2,2,2,2,2,3,3,2,2,2,2,3,2,1,1,3,2,2,1,2,3,1,3,1,3,2,1,2,3,2,2,1,3,2,2,2,3,2,2,2,1,2,1,4,1,3,4,2,3,2,1,3,1,2,1,3,3,3,2,2,2,2,2,1,2
1,1,2,2,2,1,2,32,2,1,1,2,2,1,2,2,2,1,2,3,2,2,1,1,2,1,1,2,1,1,1,1,2,2,1,2,2,1,1,1,1,2,1,1,2,3,2,2,2,1,1,1,1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,2,1,3,1,1,1,1,1,1,1,1,1,1,1,2,3,3,3,3,2,1,3,3,3
2,2,5,2,2,2,1,68,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,1,1,1,1,1,1,2,1,1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,4,1,1,1,1,2,1,1,1,1,1,1,3,4,4,4,4,3,4,4,4,4
3,2,5,1,2,2,1,31,1,1,1,2,1,1,4,1,2,1,3,1,2,3,1,1,2,1,1,1,1,1,1,1,2,2,2,2,1,1,1,1,1,1,1,1,4,4,4,4,1,3,1,1,3,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,3,1,2,1,1,4,4,1,1,4,3,4,3,3,2,3,3,3,1
4,1,2,2,2,1,2,32,2,3,4,3,2,3,3,3,1,2,4,3,3,4,3,4,4,1,1,1,1,2,2,1,4,2,4,4,2,1,1,1,1,4,4,4,2,1,2,2,1,1,1,1,1,1,4,2,1,4,4,4,3,1,2,4,4,4,1,1,1,1,3,1,2,1,4,2,1,2,2,2,1,2,2,2,1,2,3,2,2,2,2


In [31]:
dat_genpop.head()

,caseid,weight,sample_type,screener_1ongoing,screener_2impact,screener_3depression,screener_4anxiety,screener_5attention,consent,mood_yn,FNM_Q25_1,FNM_Q25_2,FNM_Q25_3,FNM_Q25_4,FNM_Q25_5,FNM_Q25_6,FNM_Q25_955,FNM_Q25_933,FNM_Q25_open1,mood_years,FNM_Q27_1,FNM_Q27_2,FNM_Q27_3,FNM_Q27_4,FNM_Q27_5,FNM_Q27_6,FNM_Q27_7,FNM_Q27_8,FNM_Q27_9,FNM_Q27_10,FNM_Q27_11,FNM_Q27_12,FNM_Q27_13,FNM_Q27_14,FNM_Q27_15,FNM_Q27_16,FNM_Q27_17,FNM_Q27_18,FNM_Q27_19,FNM_Q27_955,FNM_Q27_933,FNM_Q27_open1,mood_bothered,anxiety_yn,FNM_Q30_m_1,FNM_Q30_m_2,FNM_Q30_m_3,FNM_Q30_m_4,FNM_Q30_m_5,FNM_Q30_m_6,FNM_Q30_m_7,FNM_Q30_m_8,FNM_Q30_m_955,FNM_Q30_m_933,FNM_Q30_open1,anxiety_years,FNM_Q32_1,FNM_Q32_2,FNM_Q32_3,FNM_Q32_4,FNM_Q32_5,FNM_Q32_6,FNM_Q32_7,FNM_Q32_8,FNM_Q32_9,FNM_Q32_10,FNM_Q32_955,FNM_Q32_933,FNM_Q32_open1,anxiety_bothered,attention_yn,FNM_Q35_m_1,FNM_Q35_m_2,FNM_Q35_m_3,FNM_Q35_m_933,FNM_Q35_open1,attention_years,FNM_Q37_m_1,FNM_Q37_m_2,FNM_Q37_m_3,FNM_Q37_m_4,FNM_Q37_m_5,FNM_Q37_m_6,FNM_Q37_m_7,FNM_Q37_m_8,FNM_Q37_m_9,FNM_Q37_m_955,FNM_Q37_m_933,FNM_Q37_open1,attention_bothered,inattention_1,inattention_2,inattention_3,inattention_4,inattention_5,inattention_6,FNM_Q1_attn,inattention_7,inattention_8,inattention_9,hyperactivity_1,hyperactivity_2,hyperactivity_3,hyperactivity_4,hyperactivity_5,impulsivity_1,impulsivity_2,impulsivity_3,impulsivity_4,sct_1,sct_2,sct_3,sct_4,sct_5,sct_6,sct_7,sct_8,sct_9,gad_1,gad_2,gad_3,gad_4,gad_5,gad_6,gad_7,phq_1,phq_2,phq_3,phq_4,phq_5,phq_6,phq_7,phq_8,hitop157,hitop81,hitop34,hitop54,hitop243,hitop182,hitop69,hitop89,hitop50,check_moderately,hitop129,hitop265,hitop124,hitop231,hitop93,hitop67,hitop245,hitop281,hitop141,hitop40,hitop204,hitop21,hitop236,hitop280,hitop84,hitop120,hitop77,hitop92,hitop258,hitop39,hitop254,hitop215,hitop95,hitop106,hitop283,hitop16,hitop20,hitop189,hitop1,hitop136,hitop246,hitop248,hitop257,hitop114,hitop117,hitop250,hitop200,hitop160,hitop23,hitop165,hitop244,hitop9,hitop142,hitop230,hitop149,hitop247,hitop99,hitop66,hitop240,hitop222,hitop90,hitop113,hitop278,hitop203,hitop159,hitop123,hitop275,hitop268,hitop225,hitop143,hitop151,hitop181,hitop211,hitop17,hitop126,hitop5,hitop261,hitop220,check_notatall,hitop15,hitop72,hitop140,hitop109,hitop197,hitop104,todayinattention_1,todayinattention_2,todayinattention_3,todayinattention_4,todayinattention_5,todayinattention_6,todayinattention_7,todayinattention_8,todayinattention_9,todayhyperactivity_1,todayhyperactivity_2,todayhyperactivity_3,todayhyperactivity_4,todayhyperactivity_5,todayimpulsivity_1,todayimpulsivity_2,todayimpulsivity_3,todayimpulsivity_4,todaysct_1,todaysct_2,todaysct_3,todaysct_4,todaysct_5,todaysct_6,todaysct_7,todaysct_8,todaysct_9,today_na1,todaygad_1,todaygad_2,todaygad_3,todaygad_4,...,impulsivity_1_recontact,impulsivity_2_recontact,impulsivity_3_recontact,impulsivity_4_recontact,sct_1_recontact,sct_2_recontact,sct_3_recontact,sct_4_recontact,sct_5_recontact,sct_6_recontact,sct_7_recontact,sct_8_recontact,sct_9_recontact,gad_1_recontact,gad_2_recontact,gad_3_recontact,gad_4_recontact,gad_5_recontact,gad_6_recontact,gad_7_recontact,phq_1_recontact,phq_2_recontact,phq_3_recontact,phq_4_recontact,phq_5_recontact,phq_6_recontact,phq_7_recontact,phq_8_recontact,hitop157_recontact,hitop81_recontact,hitop34_recontact,hitop54_recontact,hitop243_recontact,hitop182_recontact,hitop69_recontact,hitop89_recontact,hitop50_recontact,check_moderately_recontact,hitop129_recontact,hitop265_recontact,hitop124_recontact,hitop231_recontact,hitop93_recontact,hitop67_recontact,hitop245_recontact,hitop281_recontact,hitop141_recontact,hitop40_recontact,hitop204_recontact,hitop21_recontact,hitop236_recontact,hitop280_recontact,hitop84_recontact,hitop120_recontact,hitop77_recontact,hitop92_recontact,hitop258_recontact,hitop39_recontact,hitop254_recontact,hitop215_recontact,hitop95_recontact,hitop106_recontact,hitop283_recontact,hitop16_recontact,hitop20_recontact,hitop189_recontact,hitop1_recontact,hitop136_recontact,hitop246_reconta

In [32]:
for c in dat_validation.columns:
    print(c)

gender
race
hispanic
current_txt
past_txt
current_med
age
hitop39
hitop77
hitop84
hitop92
hitop93
hitop123
hitop157
hitop182
hitop230
hitop246
hitop20
hitop34
hitop89
hitop203
hitop240
hitop248
hitop265
hitop120
hitop141
hitop243
hitop275
hitop109
hitop280
hitop283
hitop67
hitop142
hitop159
hitop189
hitop5
hitop66
hitop99
hitop181
hitop231
hitop21
hitop90
hitop95
hitop160
hitop254
hitop261
hitop268
hitop15
hitop104
hitop126
hitop211
hitop215
hitop257
hitop40
hitop50
hitop69
hitop81
hitop113
hitop136
hitop151
hitop197
hitop72
hitop140
hitop143
hitop220
hitop16
hitop165
hitop225
hitop247
hitop278
hitop1
hitop17
hitop114
hitop117
hitop124
hitop129
hitop204
hitop222
hitop236
hitop258
hitop9
hitop23
hitop54
hitop106
hitop149
hitop200
hitop244
hitop245
hitop250
hitop281


In [33]:
for c in dat_genpop.columns:
    print(c)

caseid
weight
sample_type
screener_1ongoing
screener_2impact
screener_3depression
screener_4anxiety
screener_5attention
consent
mood_yn
FNM_Q25_1
FNM_Q25_2
FNM_Q25_3
FNM_Q25_4
FNM_Q25_5
FNM_Q25_6
FNM_Q25_955
FNM_Q25_933
FNM_Q25_open1
mood_years
FNM_Q27_1
FNM_Q27_2
FNM_Q27_3
FNM_Q27_4
FNM_Q27_5
FNM_Q27_6
FNM_Q27_7
FNM_Q27_8
FNM_Q27_9
FNM_Q27_10
FNM_Q27_11
FNM_Q27_12
FNM_Q27_13
FNM_Q27_14
FNM_Q27_15
FNM_Q27_16
FNM_Q27_17
FNM_Q27_18
FNM_Q27_19
FNM_Q27_955
FNM_Q27_933
FNM_Q27_open1
mood_bothered
anxiety_yn
FNM_Q30_m_1
FNM_Q30_m_2
FNM_Q30_m_3
FNM_Q30_m_4
FNM_Q30_m_5
FNM_Q30_m_6
FNM_Q30_m_7
FNM_Q30_m_8
FNM_Q30_m_955
FNM_Q30_m_933
FNM_Q30_open1
anxiety_years
FNM_Q32_1
FNM_Q32_2
FNM_Q32_3
FNM_Q32_4
FNM_Q32_5
FNM_Q32_6
FNM_Q32_7
FNM_Q32_8
FNM_Q32_9
FNM_Q32_10
FNM_Q32_955
FNM_Q32_933
FNM_Q32_open1
anxiety_bothered
attention_yn
FNM_Q35_m_1
FNM_Q35_m_2
FNM_Q35_m_3
FNM_Q35_m_933
FNM_Q35_open1
attention_years
FNM_Q37_m_1
FNM_Q37_m_2
FNM_Q37_m_3
FNM_Q37_m_4
FNM_Q37_m_5
FNM_Q37_m_6
FNM_Q37_m_7
FNM_Q37

In [34]:
# do hitop columns match?
for c in dat_validation.columns:
    if 'hitop' in c:
        if c in dat_genpop.columns:
            print(c)
            print('matched in both dataframes')
        else:
            print(c)
            print('only in dat_validation')

hitop39
matched in both dataframes
hitop77
matched in both dataframes
hitop84
matched in both dataframes
hitop92
matched in both dataframes
hitop93
matched in both dataframes
hitop123
matched in both dataframes
hitop157
matched in both dataframes
hitop182
matched in both dataframes
hitop230
matched in both dataframes
hitop246
matched in both dataframes
hitop20
matched in both dataframes
hitop34
matched in both dataframes
hitop89
matched in both dataframes
hitop203
matched in both dataframes
hitop240
matched in both dataframes
hitop248
matched in both dataframes
hitop265
matched in both dataframes
hitop120
matched in both dataframes
hitop141
matched in both dataframes
hitop243
matched in both dataframes
hitop275
matched in both dataframes
hitop109
matched in both dataframes
hitop280
matched in both dataframes
hitop283
matched in both dataframes
hitop67
matched in both dataframes
hitop142
matched in both dataframes
hitop159
matched in both dataframes
hitop189
matched in both dataframes
h

In [35]:
# do hitop columns match?
for c in dat_genpop.columns:
    if 'hitop' in c:
        if c in dat_validation.columns:
            print(c)
            print('matched in both dataframes')
        else:
            print(c)
            print('only in dat_genpop')

hitop157
matched in both dataframes
hitop81
matched in both dataframes
hitop34
matched in both dataframes
hitop54
matched in both dataframes
hitop243
matched in both dataframes
hitop182
matched in both dataframes
hitop69
matched in both dataframes
hitop89
matched in both dataframes
hitop50
matched in both dataframes
hitop129
matched in both dataframes
hitop265
matched in both dataframes
hitop124
matched in both dataframes
hitop231
matched in both dataframes
hitop93
matched in both dataframes
hitop67
matched in both dataframes
hitop245
matched in both dataframes
hitop281
matched in both dataframes
hitop141
matched in both dataframes
hitop40
matched in both dataframes
hitop204
matched in both dataframes
hitop21
matched in both dataframes
hitop236
matched in both dataframes
hitop280
matched in both dataframes
hitop84
matched in both dataframes
hitop120
matched in both dataframes
hitop77
matched in both dataframes
hitop92
matched in both dataframes
hitop258
matched in both dataframes
hitop

## they match, great

In [36]:
## now keep only hitop columns; add a column that indicates group; stack the dfs

In [37]:
hitop_cols = []
for c in dat_validation.columns:
    if 'hitop' in c:
        hitop_cols.append(c)
hitop_cols

['hitop39',
 'hitop77',
 'hitop84',
 'hitop92',
 'hitop93',
 'hitop123',
 'hitop157',
 'hitop182',
 'hitop230',
 'hitop246',
 'hitop20',
 'hitop34',
 'hitop89',
 'hitop203',
 'hitop240',
 'hitop248',
 'hitop265',
 'hitop120',
 'hitop141',
 'hitop243',
 'hitop275',
 'hitop109',
 'hitop280',
 'hitop283',
 'hitop67',
 'hitop142',
 'hitop159',
 'hitop189',
 'hitop5',
 'hitop66',
 'hitop99',
 'hitop181',
 'hitop231',
 'hitop21',
 'hitop90',
 'hitop95',
 'hitop160',
 'hitop254',
 'hitop261',
 'hitop268',
 'hitop15',
 'hitop104',
 'hitop126',
 'hitop211',
 'hitop215',
 'hitop257',
 'hitop40',
 'hitop50',
 'hitop69',
 'hitop81',
 'hitop113',
 'hitop136',
 'hitop151',
 'hitop197',
 'hitop72',
 'hitop140',
 'hitop143',
 'hitop220',
 'hitop16',
 'hitop165',
 'hitop225',
 'hitop247',
 'hitop278',
 'hitop1',
 'hitop17',
 'hitop114',
 'hitop117',
 'hitop124',
 'hitop129',
 'hitop204',
 'hitop222',
 'hitop236',
 'hitop258',
 'hitop9',
 'hitop23',
 'hitop54',
 'hitop106',
 'hitop149',
 'hitop200',
 'h

In [38]:
data_validation = dat_validation[hitop_cols]
data_validation.head(3)
data_validation.shape

(496, 83)

In [39]:
data_validation = data_validation.assign(whichdata=['validation']*496)

In [40]:
data_validation.head()

,hitop39,hitop77,hitop84,hitop92,hitop93,hitop123,hitop157,hitop182,hitop230,hitop246,hitop20,hitop34,hitop89,hitop203,hitop240,hitop248,hitop265,hitop120,hitop141,hitop243,hitop275,hitop109,hitop280,hitop283,hitop67,hitop142,hitop159,hitop189,hitop5,hitop66,hitop99,hitop181,hitop231,hitop21,hitop90,hitop95,hitop160,hitop254,hitop261,hitop268,hitop15,hitop104,hitop126,hitop211,hitop215,hitop257,hitop40,hitop50,hitop69,hitop81,hitop113,hitop136,hitop151,hitop197,hitop72,hitop140,hitop143,hitop220,hitop16,hitop165,hitop225,hitop247,hitop278,hitop1,hitop17,hitop114,hitop117,hitop124,hitop129,hitop204,hitop222,hitop236,hitop258,hitop9,hitop23,hitop54,hitop106,hitop149,hitop200,hitop244,hitop245,hitop250,hitop281,whichdata
0,4,3,3,3,4,1,3,2,2,2,3,3,2,2,2,1,2,1,2,2,2,2,2,3,3,2,2,2,2,3,2,1,1,3,2,2,1,2,3,1,3,1,3,2,1,2,3,2,2,1,3,2,2,2,3,2,2,2,1,2,1,4,1,3,4,2,3,2,1,3,1,2,1,3,3,3,2,2,2,2,2,1,2,validation
1,2,1,1,2,2,1,2,2,2,1,2,3,2,2,1,1,2,1,1,2,1,1,1,1,2,2,1,2,2,1,1,1,1,2,1,1,2,3,2,2,2,1,1,1,1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,2,1,3,1,1,1,1,1,1,1,1,1,1,1,2,3,3,3,3,2,1,3,3,3,validation
2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,1,1,1,1,1,1,2,1,1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,4,1,1,1,1,2,1,1,1,1,1,1,3,4,4,4,4,3,4,4,4,4,validation
3,1,1,1,2,1,1,4,1,2,1,3,1,2,3,1,1,2,1,1,1,1,1,1,1,2,2,2,2,1,1,1,1,1,1,1,1,4,4,4,4,1,3,1,1,3,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,3,1,2,1,1,4,4,1,1,4,3,4,3,3,2,3,3,3,1,validation
4,2,3,4,3,2,3,3,3,1,2,4,3,3,4,3,4,4,1,1,1,1,2,2,1,4,2,4,4,2,1,1,1,1,4,4,4,2,1,2,2,1,1,1,1,1,1,4,2,1,4,4,4,3,1,2,4,4,4,1,1,1,1,3,1,2,1,4,2,1,2,2,2,1,2,2,2,1,2,3,2,2,2,2,validation


In [41]:
data_genpop = dat_genpop[hitop_cols]
data_genpop.shape

(500, 83)

In [42]:
data_genpop = data_genpop.assign(whichdata=['genpop']*500)
data_genpop.head()

,hitop39,hitop77,hitop84,hitop92,hitop93,hitop123,hitop157,hitop182,hitop230,hitop246,hitop20,hitop34,hitop89,hitop203,hitop240,hitop248,hitop265,hitop120,hitop141,hitop243,hitop275,hitop109,hitop280,hitop283,hitop67,hitop142,hitop159,hitop189,hitop5,hitop66,hitop99,hitop181,hitop231,hitop21,hitop90,hitop95,hitop160,hitop254,hitop261,hitop268,hitop15,hitop104,hitop126,hitop211,hitop215,hitop257,hitop40,hitop50,hitop69,hitop81,hitop113,hitop136,hitop151,hitop197,hitop72,hitop140,hitop143,hitop220,hitop16,hitop165,hitop225,hitop247,hitop278,hitop1,hitop17,hitop114,hitop117,hitop124,hitop129,hitop204,hitop222,hitop236,hitop258,hitop9,hitop23,hitop54,hitop106,hitop149,hitop200,hitop244,hitop245,hitop250,hitop281,whichdata
0,1,1,1,1,1,1,2,2,1,1,2,3,2,4,2,2,3,1,3,3,2,1,1,1,2,2,2,4,1,1,1,1,1,2,2,2,1,1,2,1,1,1,1,1,1,1,1,1,2,2,1,2,2,1,1,2,2,1,1,1,1,1,1,2,1,1,2,1,2,2,1,2,1,4,4,4,3,4,3,4,4,4,4,genpop
1,1,4,1,2,2,1,2,3,2,2,4,4,3,1,2,2,4,2,4,1,2,2,2,1,3,2,3,2,2,3,2,2,1,2,2,2,3,2,2,1,3,3,3,4,1,4,2,1,1,1,2,1,2,1,1,2,2,2,3,4,2,4,2,3,2,2,2,3,2,4,4,3,2,4,4,4,4,4,4,4,4,1,4,genpop
2,2,3,3,2,3,3,2,2,3,2,2,3,3,3,3,3,2,2,2,3,3,2,3,4,2,3,2,2,2,2,2,2,2,2,3,2,3,2,2,2,3,1,2,3,3,2,3,2,2,2,3,1,2,1,2,3,2,4,3,4,3,2,2,3,3,2,4,4,2,2,2,3,3,2,2,3,4,2,2,3,2,3,3,genpop
3,1,1,1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,2,2,2,1,1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,2,2,2,4,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,4,1,2,1,1,2,1,1,1,1,1,1,1,1,3,3,3,1,3,1,4,3,1,1,genpop
4,4,3,2,2,3,1,4,4,1,2,2,2,1,3,1,3,1,4,4,4,4,1,1,1,3,4,3,3,1,1,3,1,1,2,2,2,4,4,4,2,2,3,2,2,2,4,3,2,4,1,1,1,3,1,2,2,2,2,1,1,2,2,1,2,2,2,2,2,3,2,1,3,2,2,2,4,1,1,3,1,2,1,1,genpop


In [43]:
mydata = pd.concat([data_validation, data_genpop])
mydata.shape

(996, 84)

In [44]:
mydata.head()

,hitop39,hitop77,hitop84,hitop92,hitop93,hitop123,hitop157,hitop182,hitop230,hitop246,hitop20,hitop34,hitop89,hitop203,hitop240,hitop248,hitop265,hitop120,hitop141,hitop243,hitop275,hitop109,hitop280,hitop283,hitop67,hitop142,hitop159,hitop189,hitop5,hitop66,hitop99,hitop181,hitop231,hitop21,hitop90,hitop95,hitop160,hitop254,hitop261,hitop268,hitop15,hitop104,hitop126,hitop211,hitop215,hitop257,hitop40,hitop50,hitop69,hitop81,hitop113,hitop136,hitop151,hitop197,hitop72,hitop140,hitop143,hitop220,hitop16,hitop165,hitop225,hitop247,hitop278,hitop1,hitop17,hitop114,hitop117,hitop124,hitop129,hitop204,hitop222,hitop236,hitop258,hitop9,hitop23,hitop54,hitop106,hitop149,hitop200,hitop244,hitop245,hitop250,hitop281,whichdata
0,4,3,3,3,4,1,3,2,2,2,3,3,2,2,2,1,2,1,2,2,2,2,2,3,3,2,2,2,2,3,2,1,1,3,2,2,1,2,3,1,3,1,3,2,1,2,3,2,2,1,3,2,2,2,3,2,2,2,1,2,1,4,1,3,4,2,3,2,1,3,1,2,1,3,3,3,2,2,2,2,2,1,2,validation
1,2,1,1,2,2,1,2,2,2,1,2,3,2,2,1,1,2,1,1,2,1,1,1,1,2,2,1,2,2,1,1,1,1,2,1,1,2,3,2,2,2,1,1,1,1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,2,1,3,1,1,1,1,1,1,1,1,1,1,1,2,3,3,3,3,2,1,3,3,3,validation
2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,1,1,1,1,1,1,2,1,1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,4,1,1,1,1,2,1,1,1,1,1,1,3,4,4,4,4,3,4,4,4,4,validation
3,1,1,1,2,1,1,4,1,2,1,3,1,2,3,1,1,2,1,1,1,1,1,1,1,2,2,2,2,1,1,1,1,1,1,1,1,4,4,4,4,1,3,1,1,3,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,3,1,2,1,1,4,4,1,1,4,3,4,3,3,2,3,3,3,1,validation
4,2,3,4,3,2,3,3,3,1,2,4,3,3,4,3,4,4,1,1,1,1,2,2,1,4,2,4,4,2,1,1,1,1,4,4,4,2,1,2,2,1,1,1,1,1,1,4,2,1,4,4,4,3,1,2,4,4,4,1,1,1,1,3,1,2,1,4,2,1,2,2,2,1,2,2,2,1,2,3,2,2,2,2,validation


# Import enriched

In [45]:
datapath_enriched = '../data/NIMH0007_mental_health_num_OUTPUT.csv'
dat_enriched = pd.read_csv(datapath_enriched, dtype={'caseid':str}, engine='python')
dat_enriched.head()

,caseid,weight,sample_type,screener_1,screener_2,screener_3_1,screener_3_2,screener_3_3,consent,FNM_Q24,FNM_Q25_1,FNM_Q25_2,FNM_Q25_3,FNM_Q25_4,FNM_Q25_5,FNM_Q25_6,FNM_Q25_955,FNM_Q25_933,FNM_Q25_open1,FNM_Q26_d,FNM_Q27_1,FNM_Q27_2,FNM_Q27_3,FNM_Q27_4,FNM_Q27_5,FNM_Q27_6,FNM_Q27_7,FNM_Q27_8,FNM_Q27_9,FNM_Q27_10,FNM_Q27_11,FNM_Q27_12,FNM_Q27_13,FNM_Q27_14,FNM_Q27_15,FNM_Q27_16,FNM_Q27_17,FNM_Q27_18,FNM_Q27_19,FNM_Q27_955,FNM_Q27_933,FNM_Q27_open1,FNM_Q28,FNM_Q29,FNM_Q30_m_1,FNM_Q30_m_2,FNM_Q30_m_3,FNM_Q30_m_4,FNM_Q30_m_5,FNM_Q30_m_6,FNM_Q30_m_7,FNM_Q30_m_8,FNM_Q30_m_955,FNM_Q30_m_933,FNM_Q30_open1,FNM_Q31_d,FNM_Q32_1,FNM_Q32_2,FNM_Q32_3,FNM_Q32_4,FNM_Q32_5,FNM_Q32_6,FNM_Q32_7,FNM_Q32_8,FNM_Q32_9,FNM_Q32_10,FNM_Q32_955,FNM_Q32_933,FNM_Q32_open1,FNM_Q33,FNM_34,FNM_Q35_m_1,FNM_Q35_m_2,FNM_Q35_m_3,FNM_Q35_m_933,FNM_Q35_open1,FNM_Q36_d,FNM_Q37_m_1,FNM_Q37_m_2,FNM_Q37_m_3,FNM_Q37_m_4,FNM_Q37_m_5,FNM_Q37_m_6,FNM_Q37_m_7,FNM_Q37_m_8,FNM_Q37_m_9,FNM_Q37_m_955,FNM_Q37_m_933,FNM_Q37_open1,FNM_Q38,FNM_Q1_1,FNM_Q1_2,FNM_Q1_3,FNM_Q1_4,FNM_Q1_5,FNM_Q1_6,FNM_Q1_attn,FNM_Q1_7,FNM_Q1_8,FNM_Q1_9,FNM_Q2_1,FNM_Q2_2,FNM_Q2_3,FNM_Q2_4,FNM_Q2_5,FNM_Q3_1,FNM_Q3_2,FNM_Q3_3,FNM_Q3_4,FNM_Q4_1,FNM_Q4_2,FNM_Q4_3,FNM_Q4_4,FNM_Q4_5,FNM_Q4_6,FNM_Q4_7,FNM_Q4_8,FNM_Q4_9,FNM_Q5_1,FNM_Q5_2,FNM_Q5_3,FNM_Q5_4,FNM_Q5_5,FNM_Q5_6,FNM_Q5_7,FNM_Q6_1,FNM_Q6_2,FNM_Q6_3,FNM_Q6_4,FNM_Q6_5,FNM_Q6_6,FNM_Q6_7,FNM_Q6_8,FNM_Q7_1,FNM_Q7_2,FNM_Q7_3,FNM_Q7_4,FNM_Q7_5,FNM_Q8_1,FNM_Q8_2,FNM_Q8_3,FNM_Q8_4,FNM_Q8_5,FNM_Q8_6,FNM_Q9_1,FNM_Q9_2,FNM_Q9_3,FNM_Q9_4,FNM_Q9_5,FNM_Q10_1,FNM_Q10_2,FNM_Q10_3,FNM_Q10_4,FNM_Q10_5,FNM_Q11_1,FNM_Q11_2,FNM_Q11_3,FNM_Q11_4,FNM_Q11_5,FNM_Q12_1,FNM_Q12_2,FNM_Q12_3,FNM_Q12_4,FNM_Q12_5,FNM_Q13_1,FNM_Q13_2,FNM_Q13_3,FNM_Q13_4,FNM_Q13_5,FNM_Q14_1,FNM_Q14_2,FNM_Q14_3,FNM_Q14_4,FNM_Q14_5,FNM_Q15_1,FNM_Q15_2,FNM_Q15_3,FNM_Q15_4,FNM_Q15_5,FNM_Q16_1,FNM_Q16_2,FNM_Q16_3,FNM_Q16_4,FNM_Q16_5,FNM_Q17_1,FNM_Q17_2,FNM_Q17_3,FNM_Q17_4,FNM_Q17_5,FNM_Q18_1,FNM_Q18_2,FNM_Q18_3,FNM_Q18_4,FNM_Q18_5,FNM_Q19_1,FNM_Q19_2,FNM_Q19_3,FNM_Q19_4,FNM_Q19_5,FNM_Q20_1,FNM_Q20_2,FNM_Q20_3,FNM_Q20_4,FNM_Q20_5,FNM_Q21_1,FNM_Q21_2,FNM_Q21_3,FNM_Q21_4,FNM_Q21_5,FNM_Q22_1,FNM_Q22_2,FNM_Q22_3,FNM_Q22_4,FNM_Q22_5,FNM_Q22_6,FNM_Q23_1,FNM_Q23_2,FNM_Q23_3,FNM_Q40_m_1,FNM_Q40_m_2,FNM_Q40_m_3,FNM_Q40_m_4,FNM_Q40_m_5,FNM_Q40_m_6,FNM_Q40_m_7,FNM_Q40_m_8,FNM_Q40_m_9,FNM_Q40_m_10,FNM_Q40_m_11,FNM_Q40_m_12,FNM_Q40_m_13,FNM_Q40_m_14,FNM_Q40_m_15,FNM_Q40_m_16,FNM_Q40_m_17,FNM_Q40_m_18,FNM_Q40_m_19,FNM_Q40_m_20,FNM_Q40_m_21,FNM_Q40_m_22,FNM_Q40_m_23,FNM_Q40_m_24,FNM_Q40_m_25,FNM_Q40_m_26,FNM_Q40_m_27,FNM_Q40_m_944,FNM_Q41_m_1,FNM_Q41_m_2,FNM_Q41_m_3,FNM_Q41_m_4,...,FNM_Q3_1_recontact,FNM_Q3_2_recontact,FNM_Q3_3_recontact,FNM_Q3_4_recontact,FNM_Q4_1_recontact,FNM_Q4_2_recontact,FNM_Q4_3_recontact,FNM_Q4_4_recontact,FNM_Q4_5_recontact,FNM_Q4_6_recontact,FNM_Q4_7_recontact,FNM_Q4_8_recontact,FNM_Q4_9_recontact,FNM_Q5_1_recontact,FNM_Q5_2_recontact,FNM_Q5_3_recontact,FNM_Q5_4_recontact,FNM_Q5_5_recontact,FNM_Q5_6_recontact,FNM_Q5_7_recontact,FNM_Q6_1_recontact,FNM_Q6_2_recontact,FNM_Q6_3_recontact,FNM_Q6_4_recontact,FNM_Q6_5_recontact,FNM_Q6_6_recontact,FNM_Q6_7_recontact,FNM_Q6_8_recontact,FNM_Q7_1_recontact,FNM_Q7_2_recontact,FNM_Q7_3_recontact,FNM_Q7_4_recontact,FNM_Q7_5_recontact,FNM_Q8_1_recontact,FNM_Q8_2_recontact,FNM_Q8_3_recontact,FNM_Q8_4_recontact,FNM_Q8_5_recontact,FNM_Q8_6_recontact,FNM_Q9_1_recontact,FNM_Q9_2_recontact,FNM_Q9_3_recontact,FNM_Q9_4_recontact,FNM_Q9_5_recontact,FNM_Q10_1_recontact,FNM_Q10_2_recontact,FNM_Q10_3_recontact,FNM_Q10_4_recontact,FNM_Q10_5_recontact,FNM_Q11_1_recontact,FNM_Q11_2_recontact,FNM_Q11_3_recontact,FNM_Q11_4_recontact,FNM_Q11_5_recontact,FNM_Q12_1_recontact,FNM_Q12_2_recontact,FNM_Q12_3_recontact,FNM_Q12_4_recontact,FNM_Q12_5_recontact,FNM_Q13_1_recontact,FNM_Q13_2_recontact,FNM_Q13_3_recontact,FNM_Q13_4_recontact,FNM_Q13_5_recontact,FNM_Q14_1_recontact,FNM_Q14_2_recontact,FNM_Q14_3_recontact,FNM_Q14_4_recontact,FNM_Q14_5_recontact,FNM_Q15_1_recontact,FNM_Q15_2_re

In [46]:
dat.head()

,caseid,weight,sample_type,screener_1ongoing,screener_2impact,screener_3depression,screener_4anxiety,screener_5attention,consent,mood_yn,FNM_Q25_1,FNM_Q25_2,FNM_Q25_3,FNM_Q25_4,FNM_Q25_5,FNM_Q25_6,FNM_Q25_955,FNM_Q25_933,FNM_Q25_open1,mood_years,FNM_Q27_1,FNM_Q27_2,FNM_Q27_3,FNM_Q27_4,FNM_Q27_5,FNM_Q27_6,FNM_Q27_7,FNM_Q27_8,FNM_Q27_9,FNM_Q27_10,FNM_Q27_11,FNM_Q27_12,FNM_Q27_13,FNM_Q27_14,FNM_Q27_15,FNM_Q27_16,FNM_Q27_17,FNM_Q27_18,FNM_Q27_19,FNM_Q27_955,FNM_Q27_933,FNM_Q27_open1,mood_bothered,anxiety_yn,FNM_Q30_m_1,FNM_Q30_m_2,FNM_Q30_m_3,FNM_Q30_m_4,FNM_Q30_m_5,FNM_Q30_m_6,FNM_Q30_m_7,FNM_Q30_m_8,FNM_Q30_m_955,FNM_Q30_m_933,FNM_Q30_open1,anxiety_years,FNM_Q32_1,FNM_Q32_2,FNM_Q32_3,FNM_Q32_4,FNM_Q32_5,FNM_Q32_6,FNM_Q32_7,FNM_Q32_8,FNM_Q32_9,FNM_Q32_10,FNM_Q32_955,FNM_Q32_933,FNM_Q32_open1,anxiety_bothered,attention_yn,FNM_Q35_m_1,FNM_Q35_m_2,FNM_Q35_m_3,FNM_Q35_m_933,FNM_Q35_open1,attention_years,FNM_Q37_m_1,FNM_Q37_m_2,FNM_Q37_m_3,FNM_Q37_m_4,FNM_Q37_m_5,FNM_Q37_m_6,FNM_Q37_m_7,FNM_Q37_m_8,FNM_Q37_m_9,FNM_Q37_m_955,FNM_Q37_m_933,FNM_Q37_open1,attention_bothered,inattention_1,inattention_2,inattention_3,inattention_4,inattention_5,inattention_6,FNM_Q1_attn,inattention_7,inattention_8,inattention_9,hyperactivity_1,hyperactivity_2,hyperactivity_3,hyperactivity_4,hyperactivity_5,impulsivity_1,impulsivity_2,impulsivity_3,impulsivity_4,sct_1,sct_2,sct_3,sct_4,sct_5,sct_6,sct_7,sct_8,sct_9,gad_1,gad_2,gad_3,gad_4,gad_5,gad_6,gad_7,phq_1,phq_2,phq_3,phq_4,phq_5,phq_6,phq_7,phq_8,hitop157,hitop81,hitop34,hitop54,hitop243,hitop182,hitop69,hitop89,hitop50,check_moderately,hitop129,hitop265,hitop124,hitop231,hitop93,hitop67,hitop245,hitop281,hitop141,hitop40,hitop204,hitop21,hitop236,hitop280,hitop84,hitop120,hitop77,hitop92,hitop258,hitop39,hitop254,hitop215,hitop95,hitop106,hitop283,hitop16,hitop20,hitop189,hitop1,hitop136,hitop246,hitop248,hitop257,hitop114,hitop117,hitop250,hitop200,hitop160,hitop23,hitop165,hitop244,hitop9,hitop142,hitop230,hitop149,hitop247,hitop99,hitop66,hitop240,hitop222,hitop90,hitop113,hitop278,hitop203,hitop159,hitop123,hitop275,hitop268,hitop225,hitop143,hitop151,hitop181,hitop211,hitop17,hitop126,hitop5,hitop261,hitop220,check_notatall,hitop15,hitop72,hitop140,hitop109,hitop197,hitop104,todayinattention_1,todayinattention_2,todayinattention_3,todayinattention_4,todayinattention_5,todayinattention_6,todayinattention_7,todayinattention_8,todayinattention_9,todayhyperactivity_1,todayhyperactivity_2,todayhyperactivity_3,todayhyperactivity_4,todayhyperactivity_5,todayimpulsivity_1,todayimpulsivity_2,todayimpulsivity_3,todayimpulsivity_4,todaysct_1,todaysct_2,todaysct_3,todaysct_4,todaysct_5,todaysct_6,todaysct_7,todaysct_8,todaysct_9,today_na1,todaygad_1,todaygad_2,todaygad_3,todaygad_4,...,impulsivity_1_recontact,impulsivity_2_recontact,impulsivity_3_recontact,impulsivity_4_recontact,sct_1_recontact,sct_2_recontact,sct_3_recontact,sct_4_recontact,sct_5_recontact,sct_6_recontact,sct_7_recontact,sct_8_recontact,sct_9_recontact,gad_1_recontact,gad_2_recontact,gad_3_recontact,gad_4_recontact,gad_5_recontact,gad_6_recontact,gad_7_recontact,phq_1_recontact,phq_2_recontact,phq_3_recontact,phq_4_recontact,phq_5_recontact,phq_6_recontact,phq_7_recontact,phq_8_recontact,hitop157_recontact,hitop81_recontact,hitop34_recontact,hitop54_recontact,hitop243_recontact,hitop182_recontact,hitop69_recontact,hitop89_recontact,hitop50_recontact,check_moderately_recontact,hitop129_recontact,hitop265_recontact,hitop124_recontact,hitop231_recontact,hitop93_recontact,hitop67_recontact,hitop245_recontact,hitop281_recontact,hitop141_recontact,hitop40_recontact,hitop204_recontact,hitop21_recontact,hitop236_recontact,hitop280_recontact,hitop84_recontact,hitop120_recontact,hitop77_recontact,hitop92_recontact,hitop258_recontact,hitop39_recontact,hitop254_recontact,hitop215_recontact,hitop95_recontact,hitop106_recontact,hitop283_recontact,hitop16_recontact,hitop20_recontact,hitop189_recontact,hitop1_recontact,hitop136_recontact,hitop246_reconta

In [47]:
dat.columns

Index(['caseid', 'weight', 'sample_type', 'screener_1ongoing',
       'screener_2impact', 'screener_3depression', 'screener_4anxiety',
       'screener_5attention', 'consent', 'mood_yn',
       ...
       'todayhitop140_recontact', 'todayhitop109_recontact',
       'todayhitop197_recontact', 'todayhitop104_recontact',
       'today_na5_recontact', 'family_wealth_recontact', 'WHODAS_H2_recontact',
       'WHODAS_H3_recontact', 'starttime_recontact', 'endtime_recontact'],
      dtype='object', length=728)

In [48]:
dat_enriched.columns

Index(['caseid', 'weight', 'sample_type', 'screener_1', 'screener_2',
       'screener_3_1', 'screener_3_2', 'screener_3_3', 'consent', 'FNM_Q24',
       ...
       'FNM_44_m_31_recontact', 'FNM_44_m_32_recontact',
       'FNM_44_m_33_recontact', 'FNM_44_m_34_recontact',
       'FNM_44_m_35_recontact', 'family_wealth_recontact',
       'WHODAS_H2_recontact', 'WHODAS_H3_recontact', 'starttime_recontact',
       'endtime_recontact'],
      dtype='object', length=728)

In [49]:
# enriched and genpop have a very similar format, good!

In [50]:
dat_enriched = dat_enriched.rename(ename_lut, axis=1)
dat_enriched.head(3)

,caseid,weight,sample_type,screener_1ongoing,screener_2impact,screener_3depression,screener_4anxiety,screener_5attention,consent,mood_yn,FNM_Q25_1,FNM_Q25_2,FNM_Q25_3,FNM_Q25_4,FNM_Q25_5,FNM_Q25_6,FNM_Q25_955,FNM_Q25_933,FNM_Q25_open1,mood_years,FNM_Q27_1,FNM_Q27_2,FNM_Q27_3,FNM_Q27_4,FNM_Q27_5,FNM_Q27_6,FNM_Q27_7,FNM_Q27_8,FNM_Q27_9,FNM_Q27_10,FNM_Q27_11,FNM_Q27_12,FNM_Q27_13,FNM_Q27_14,FNM_Q27_15,FNM_Q27_16,FNM_Q27_17,FNM_Q27_18,FNM_Q27_19,FNM_Q27_955,FNM_Q27_933,FNM_Q27_open1,mood_bothered,anxiety_yn,FNM_Q30_m_1,FNM_Q30_m_2,FNM_Q30_m_3,FNM_Q30_m_4,FNM_Q30_m_5,FNM_Q30_m_6,FNM_Q30_m_7,FNM_Q30_m_8,FNM_Q30_m_955,FNM_Q30_m_933,FNM_Q30_open1,anxiety_years,FNM_Q32_1,FNM_Q32_2,FNM_Q32_3,FNM_Q32_4,FNM_Q32_5,FNM_Q32_6,FNM_Q32_7,FNM_Q32_8,FNM_Q32_9,FNM_Q32_10,FNM_Q32_955,FNM_Q32_933,FNM_Q32_open1,anxiety_bothered,attention_yn,FNM_Q35_m_1,FNM_Q35_m_2,FNM_Q35_m_3,FNM_Q35_m_933,FNM_Q35_open1,attention_years,FNM_Q37_m_1,FNM_Q37_m_2,FNM_Q37_m_3,FNM_Q37_m_4,FNM_Q37_m_5,FNM_Q37_m_6,FNM_Q37_m_7,FNM_Q37_m_8,FNM_Q37_m_9,FNM_Q37_m_955,FNM_Q37_m_933,FNM_Q37_open1,attention_bothered,inattention_1,inattention_2,inattention_3,inattention_4,inattention_5,inattention_6,FNM_Q1_attn,inattention_7,inattention_8,inattention_9,hyperactivity_1,hyperactivity_2,hyperactivity_3,hyperactivity_4,hyperactivity_5,impulsivity_1,impulsivity_2,impulsivity_3,impulsivity_4,sct_1,sct_2,sct_3,sct_4,sct_5,sct_6,sct_7,sct_8,sct_9,gad_1,gad_2,gad_3,gad_4,gad_5,gad_6,gad_7,phq_1,phq_2,phq_3,phq_4,phq_5,phq_6,phq_7,phq_8,hitop157,hitop81,hitop34,hitop54,hitop243,hitop182,hitop69,hitop89,hitop50,check_moderately,hitop129,hitop265,hitop124,hitop231,hitop93,hitop67,hitop245,hitop281,hitop141,hitop40,hitop204,hitop21,hitop236,hitop280,hitop84,hitop120,hitop77,hitop92,hitop258,hitop39,hitop254,hitop215,hitop95,hitop106,hitop283,hitop16,hitop20,hitop189,hitop1,hitop136,hitop246,hitop248,hitop257,hitop114,hitop117,hitop250,hitop200,hitop160,hitop23,hitop165,hitop244,hitop9,hitop142,hitop230,hitop149,hitop247,hitop99,hitop66,hitop240,hitop222,hitop90,hitop113,hitop278,hitop203,hitop159,hitop123,hitop275,hitop268,hitop225,hitop143,hitop151,hitop181,hitop211,hitop17,hitop126,hitop5,hitop261,hitop220,check_notatall,hitop15,hitop72,hitop140,hitop109,hitop197,hitop104,todayinattention_1,todayinattention_2,todayinattention_3,todayinattention_4,todayinattention_5,todayinattention_6,todayinattention_7,todayinattention_8,todayinattention_9,todayhyperactivity_1,todayhyperactivity_2,todayhyperactivity_3,todayhyperactivity_4,todayhyperactivity_5,todayimpulsivity_1,todayimpulsivity_2,todayimpulsivity_3,todayimpulsivity_4,todaysct_1,todaysct_2,todaysct_3,todaysct_4,todaysct_5,todaysct_6,todaysct_7,todaysct_8,todaysct_9,today_na1,todaygad_1,todaygad_2,todaygad_3,todaygad_4,...,impulsivity_1_recontact,impulsivity_2_recontact,impulsivity_3_recontact,impulsivity_4_recontact,sct_1_recontact,sct_2_recontact,sct_3_recontact,sct_4_recontact,sct_5_recontact,sct_6_recontact,sct_7_recontact,sct_8_recontact,sct_9_recontact,gad_1_recontact,gad_2_recontact,gad_3_recontact,gad_4_recontact,gad_5_recontact,gad_6_recontact,gad_7_recontact,phq_1_recontact,phq_2_recontact,phq_3_recontact,phq_4_recontact,phq_5_recontact,phq_6_recontact,phq_7_recontact,phq_8_recontact,hitop157_recontact,hitop81_recontact,hitop34_recontact,hitop54_recontact,hitop243_recontact,hitop182_recontact,hitop69_recontact,hitop89_recontact,hitop50_recontact,check_moderately_recontact,hitop129_recontact,hitop265_recontact,hitop124_recontact,hitop231_recontact,hitop93_recontact,hitop67_recontact,hitop245_recontact,hitop281_recontact,hitop141_recontact,hitop40_recontact,hitop204_recontact,hitop21_recontact,hitop236_recontact,hitop280_recontact,hitop84_recontact,hitop120_recontact,hitop77_recontact,hitop92_recontact,hitop258_recontact,hitop39_recontact,hitop254_recontact,hitop215_recontact,hitop95_recontact,hitop106_recontact,hitop283_recontact,hitop16_recontact,hitop20_recontact,hitop189_recontact,hitop1_recontact,hitop136_recontact,hitop246_reconta

In [51]:
# do columns match?

In [52]:
# 1
# do hitop columns match?
for c in dat_validation.columns:
    if 'hitop' in c:
        if c in dat_enriched.columns:
            print(c)
            print('matched in both dataframes')
        else:
            print(c)
            print('only in dat_validation')

hitop39
matched in both dataframes
hitop77
matched in both dataframes
hitop84
matched in both dataframes
hitop92
matched in both dataframes
hitop93
matched in both dataframes
hitop123
matched in both dataframes
hitop157
matched in both dataframes
hitop182
matched in both dataframes
hitop230
matched in both dataframes
hitop246
matched in both dataframes
hitop20
matched in both dataframes
hitop34
matched in both dataframes
hitop89
matched in both dataframes
hitop203
matched in both dataframes
hitop240
matched in both dataframes
hitop248
matched in both dataframes
hitop265
matched in both dataframes
hitop120
matched in both dataframes
hitop141
matched in both dataframes
hitop243
matched in both dataframes
hitop275
matched in both dataframes
hitop109
matched in both dataframes
hitop280
matched in both dataframes
hitop283
matched in both dataframes
hitop67
matched in both dataframes
hitop142
matched in both dataframes
hitop159
matched in both dataframes
hitop189
matched in both dataframes
h

In [53]:
# 2
# do hitop columns match?
for c in dat_enriched.columns:
    if 'hitop' in c:
        if c in dat_validation.columns:
            print(c)
            print('matched in both dataframes')
        else:
            print(c)
            print('only in dat_enriched')

hitop157
matched in both dataframes
hitop81
matched in both dataframes
hitop34
matched in both dataframes
hitop54
matched in both dataframes
hitop243
matched in both dataframes
hitop182
matched in both dataframes
hitop69
matched in both dataframes
hitop89
matched in both dataframes
hitop50
matched in both dataframes
hitop129
matched in both dataframes
hitop265
matched in both dataframes
hitop124
matched in both dataframes
hitop231
matched in both dataframes
hitop93
matched in both dataframes
hitop67
matched in both dataframes
hitop245
matched in both dataframes
hitop281
matched in both dataframes
hitop141
matched in both dataframes
hitop40
matched in both dataframes
hitop204
matched in both dataframes
hitop21
matched in both dataframes
hitop236
matched in both dataframes
hitop280
matched in both dataframes
hitop84
matched in both dataframes
hitop120
matched in both dataframes
hitop77
matched in both dataframes
hitop92
matched in both dataframes
hitop258
matched in both dataframes
hitop

In [54]:
# today stuff is only in enriched, let's drop them

In [55]:
todaycols = []
recontactcols = []
for c in dat_enriched.columns:
    if 'today' in c:
        todaycols.append(c)
    if 'recontact' in c:
        recontactcols.append(c)

In [56]:
dat_enriched_withrecontact = dat_enriched.drop(columns = todaycols)
dat_enriched = dat_enriched_withrecontact.drop(columns = recontactcols)

KeyError: "['todayinattention_1_recontact', 'todayinattention_2_recontact', 'todayinattention_3_recontact', 'todayinattention_4_recontact', 'todayinattention_5_recontact', 'todayinattention_6_recontact', 'todayinattention_7_recontact', 'todayinattention_8_recontact', 'todayinattention_9_recontact', 'todayhyperactivity_1_recontact', 'todayhyperactivity_2_recontact', 'todayhyperactivity_3_recontact', 'todayhyperactivity_4_recontact', 'todayhyperactivity_5_recontact', 'todayimpulsivity_1_recontact', 'todayimpulsivity_2_recontact', 'todayimpulsivity_3_recontact', 'todayimpulsivity_4_recontact', 'todaysct_1_recontact', 'todaysct_2_recontact', 'todaysct_3_recontact', 'todaysct_4_recontact', 'todaysct_5_recontact', 'todaysct_6_recontact', 'todaysct_7_recontact', 'todaysct_8_recontact', 'todaysct_9_recontact', 'today_na1_recontact', 'todaygad_1_recontact', 'todaygad_2_recontact', 'todaygad_3_recontact', 'todaygad_4_recontact', 'todaygad_5_recontact', 'todaygad_6_recontact', 'todaygad_7_reconta

In [ ]:
# double-check
# do hitop columns match?
for c in dat_enriched.columns:
    if 'hitop' in c:
        if c in dat_validation.columns:
            print(c)
            print('matched in both dataframes')
        else:
            print(c)
            print('only in dat_enriched')

In [ ]:
# they match, great

In [57]:
data_enriched = dat_enriched[hitop_cols]
data_enriched.shape

(310, 83)

In [58]:
# check
data_genpop.shape

(500, 84)

In [59]:
data_enriched = data_enriched.assign(whichdata=['enriched']*310)
data_enriched.head()

,hitop39,hitop77,hitop84,hitop92,hitop93,hitop123,hitop157,hitop182,hitop230,hitop246,hitop20,hitop34,hitop89,hitop203,hitop240,hitop248,hitop265,hitop120,hitop141,hitop243,hitop275,hitop109,hitop280,hitop283,hitop67,hitop142,hitop159,hitop189,hitop5,hitop66,hitop99,hitop181,hitop231,hitop21,hitop90,hitop95,hitop160,hitop254,hitop261,hitop268,hitop15,hitop104,hitop126,hitop211,hitop215,hitop257,hitop40,hitop50,hitop69,hitop81,hitop113,hitop136,hitop151,hitop197,hitop72,hitop140,hitop143,hitop220,hitop16,hitop165,hitop225,hitop247,hitop278,hitop1,hitop17,hitop114,hitop117,hitop124,hitop129,hitop204,hitop222,hitop236,hitop258,hitop9,hitop23,hitop54,hitop106,hitop149,hitop200,hitop244,hitop245,hitop250,hitop281,whichdata
0,2,2,3,3,2,2,3,2,1,1,4,1,2,2,2,1,1,4,1,4,4,1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,4,4,4,4,1,2,1,1,3,1,2,1,4,1,4,1,4,1,2,4,3,4,1,1,4,4,2,4,4,1,4,2,1,1,1,3,1,2,2,3,1,1,1,2,4,3,2,enriched
1,4,4,4,4,4,3,4,4,2,4,4,4,4,4,4,4,4,2,2,3,4,3,4,2,4,4,4,4,4,4,2,2,4,4,4,4,3,4,4,4,1,4,4,4,4,4,4,3,1,4,4,4,4,4,4,4,4,4,3,4,4,4,4,4,4,4,4,4,4,4,4,4,4,1,1,4,1,1,1,1,4,1,1,enriched
2,2,3,3,3,3,2,3,2,2,3,4,4,3,4,4,2,3,1,1,2,2,2,2,1,4,4,4,4,3,1,4,2,4,4,4,4,2,3,1,3,2,3,3,2,2,2,2,4,1,4,3,1,4,1,3,4,3,4,1,1,2,1,2,3,2,3,4,3,2,3,2,4,3,2,2,2,2,2,1,3,2,1,1,enriched
3,4,4,2,3,1,1,4,1,1,1,2,3,1,3,2,4,2,1,1,1,4,2,1,2,2,2,2,2,1,1,1,1,1,2,2,3,4,4,4,4,3,4,2,2,3,4,1,1,1,3,3,1,1,1,2,4,4,3,1,1,1,1,1,1,2,1,3,3,1,4,2,2,3,1,2,4,1,1,1,4,4,2,4,enriched
4,2,1,1,2,1,1,3,2,1,2,2,2,1,3,1,1,1,1,1,1,1,1,1,1,3,4,3,3,1,1,3,2,1,1,3,2,1,1,1,1,2,1,2,2,1,2,1,2,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,2,2,2,2,2,1,2,2,3,3,2,4,3,4,1,2,3,2,enriched


In [60]:
mydata_valid_enriched = pd.concat([data_validation, data_enriched])
mydata_valid_enriched.shape

(806, 84)

# Prep data for original vs recontact

In [61]:
# does genpop have recontact?
dat_genpop.columns

Index(['caseid', 'weight', 'sample_type', 'screener_1ongoing',
       'screener_2impact', 'screener_3depression', 'screener_4anxiety',
       'screener_5attention', 'consent', 'mood_yn',
       ...
       'todayhitop140_recontact', 'todayhitop109_recontact',
       'todayhitop197_recontact', 'todayhitop104_recontact',
       'today_na5_recontact', 'family_wealth_recontact', 'WHODAS_H2_recontact',
       'WHODAS_H3_recontact', 'starttime_recontact', 'endtime_recontact'],
      dtype='object', length=728)

In [62]:
for c in dat_genpop.columns:
    if "recontact" in c:
        print(c)

screener_1ongoing_recontact
screener_2impact_recontact
screener_3depression_recontact
screener_4anxiety_recontact
screener_5attention_recontact
consent_recontact
mood_yn_recontact
FNM_Q25_1_recontact
FNM_Q25_2_recontact
FNM_Q25_3_recontact
FNM_Q25_4_recontact
FNM_Q25_5_recontact
FNM_Q25_6_recontact
FNM_Q25_955_recontact
FNM_Q25_933_recontact
FNM_Q25_open1_recontact
mood_years_recontact
FNM_Q27_1_recontact
FNM_Q27_2_recontact
FNM_Q27_3_recontact
FNM_Q27_4_recontact
FNM_Q27_5_recontact
FNM_Q27_6_recontact
FNM_Q27_7_recontact
FNM_Q27_8_recontact
FNM_Q27_9_recontact
FNM_Q27_10_recontact
FNM_Q27_11_recontact
FNM_Q27_12_recontact
FNM_Q27_13_recontact
FNM_Q27_14_recontact
FNM_Q27_15_recontact
FNM_Q27_16_recontact
FNM_Q27_17_recontact
FNM_Q27_18_recontact
FNM_Q27_19_recontact
FNM_Q27_955_recontact
FNM_Q27_933_recontact
FNM_Q27_open1_recontact
mood_bothered_recontact
anxiety_yn_recontact
FNM_Q30_m_1_recontact
FNM_Q30_m_2_recontact
FNM_Q30_m_3_recontact
FNM_Q30_m_4_recontact
FNM_Q30_m_5_recontac

In [63]:
# here make the if loop for which data im using
df = dat_enriched_withrecontact # or df = dat_genpop
print(df.shape)
df_rows = df.shape[0]
df_rows

(310, 464)


310

In [64]:
df_helper = df.copy(deep=True)

In [65]:
df = df.assign(whichvisit=['initialvisit']*df_rows)
df_helper = df_helper.assign(whichvisit=['recontactvisit']*df_rows)

In [66]:
# in df, drop all "recontact" columns
# in df_helper, drop all initial visit columns, then rename the recontact columns 

In [67]:
cols_recontact = []
cols_original = []
for c in df.columns:
    if "recontact" in c:
        cols_recontact.append(c)
        cols_original.append(c[:-10])
#print(cols_recontact)
#print(cols_original)

In [68]:
df = df.drop(columns=cols_recontact)
df_helper = df_helper.drop(columns=cols_original)

In [69]:
cols_dict = {}
for cname in cols_recontact:
    cols_dict[cname] = cname[:-10]
cols_dict

{'screener_1ongoing_recontact': 'screener_1ongoing',
 'screener_2impact_recontact': 'screener_2impact',
 'screener_3depression_recontact': 'screener_3depression',
 'screener_4anxiety_recontact': 'screener_4anxiety',
 'screener_5attention_recontact': 'screener_5attention',
 'consent_recontact': 'consent',
 'mood_yn_recontact': 'mood_yn',
 'FNM_Q25_1_recontact': 'FNM_Q25_1',
 'FNM_Q25_2_recontact': 'FNM_Q25_2',
 'FNM_Q25_3_recontact': 'FNM_Q25_3',
 'FNM_Q25_4_recontact': 'FNM_Q25_4',
 'FNM_Q25_5_recontact': 'FNM_Q25_5',
 'FNM_Q25_6_recontact': 'FNM_Q25_6',
 'FNM_Q25_955_recontact': 'FNM_Q25_955',
 'FNM_Q25_933_recontact': 'FNM_Q25_933',
 'FNM_Q25_open1_recontact': 'FNM_Q25_open1',
 'mood_years_recontact': 'mood_years',
 'FNM_Q27_1_recontact': 'FNM_Q27_1',
 'FNM_Q27_2_recontact': 'FNM_Q27_2',
 'FNM_Q27_3_recontact': 'FNM_Q27_3',
 'FNM_Q27_4_recontact': 'FNM_Q27_4',
 'FNM_Q27_5_recontact': 'FNM_Q27_5',
 'FNM_Q27_6_recontact': 'FNM_Q27_6',
 'FNM_Q27_7_recontact': 'FNM_Q27_7',
 'FNM_Q27_8_re

In [70]:
# rename cols in df_helper
df_helper.rename(columns=cols_dict, inplace=True)
df_helper.head(3)

,caseid,weight,sample_type,genhealth,healthinsur_srg_1,healthinsur_srg_2,healthinsur_srg_3,healthinsur_srg_4,healthinsur_srg_5,healthinsur_srg_6,healthinsur_srg_7,healthinsur_srg_8,healthinsur_srg_99,race_client,hispanic,birthyr,gender,sex,educ,marstat,employ,inputstate,faminc_new,child18,screener_1ongoing,screener_2impact,screener_3depression,screener_4anxiety,screener_5attention,consent,mood_yn,FNM_Q25_1,FNM_Q25_2,FNM_Q25_3,FNM_Q25_4,FNM_Q25_5,FNM_Q25_6,FNM_Q25_955,FNM_Q25_933,FNM_Q25_open1,mood_years,FNM_Q27_1,FNM_Q27_2,FNM_Q27_3,FNM_Q27_4,FNM_Q27_5,FNM_Q27_6,FNM_Q27_7,FNM_Q27_8,FNM_Q27_9,FNM_Q27_10,FNM_Q27_11,FNM_Q27_12,FNM_Q27_13,FNM_Q27_14,FNM_Q27_15,FNM_Q27_16,FNM_Q27_17,FNM_Q27_18,FNM_Q27_19,FNM_Q27_955,FNM_Q27_933,FNM_Q27_open1,mood_bothered,anxiety_yn,FNM_Q30_m_1,FNM_Q30_m_2,FNM_Q30_m_3,FNM_Q30_m_4,FNM_Q30_m_5,FNM_Q30_m_6,FNM_Q30_m_7,FNM_Q30_m_8,FNM_Q30_m_955,FNM_Q30_m_933,FNM_Q30_open1,anxiety_years,FNM_Q32_1,FNM_Q32_2,FNM_Q32_3,FNM_Q32_4,FNM_Q32_5,FNM_Q32_6,FNM_Q32_7,FNM_Q32_8,FNM_Q32_9,FNM_Q32_10,FNM_Q32_955,FNM_Q32_933,FNM_Q32_open1,anxiety_bothered,attention_yn,FNM_Q35_m_1,FNM_Q35_m_2,FNM_Q35_m_3,FNM_Q35_m_933,FNM_Q35_open1,attention_years,FNM_Q37_m_1,FNM_Q37_m_2,FNM_Q37_m_3,FNM_Q37_m_4,FNM_Q37_m_5,FNM_Q37_m_6,FNM_Q37_m_7,FNM_Q37_m_8,FNM_Q37_m_9,FNM_Q37_m_955,FNM_Q37_m_933,FNM_Q37_open1,attention_bothered,inattention_1,inattention_2,inattention_3,inattention_4,inattention_5,inattention_6,FNM_Q1_attn,inattention_7,inattention_8,inattention_9,hyperactivity_1,hyperactivity_2,hyperactivity_3,hyperactivity_4,hyperactivity_5,impulsivity_1,impulsivity_2,impulsivity_3,impulsivity_4,sct_1,sct_2,sct_3,sct_4,sct_5,sct_6,sct_7,sct_8,sct_9,gad_1,gad_2,gad_3,gad_4,gad_5,gad_6,gad_7,phq_1,phq_2,phq_3,phq_4,phq_5,phq_6,phq_7,phq_8,hitop157,hitop81,hitop34,hitop54,hitop243,hitop182,hitop69,hitop89,hitop50,check_moderately,hitop129,hitop265,hitop124,hitop231,hitop93,hitop67,hitop245,hitop281,hitop141,hitop40,hitop204,hitop21,hitop236,hitop280,hitop84,hitop120,hitop77,hitop92,hitop258,hitop39,hitop254,hitop215,hitop95,hitop106,hitop283,hitop16,hitop20,hitop189,hitop1,hitop136,hitop246,hitop248,hitop257,hitop114,hitop117,hitop250,hitop200,hitop160,hitop23,hitop165,hitop244,hitop9,hitop142,hitop230,hitop149,hitop247,hitop99,hitop66,hitop240,hitop222,hitop90,hitop113,hitop278,hitop203,hitop159,hitop123,hitop275,hitop268,hitop225,hitop143,hitop151,hitop181,hitop211,hitop17,hitop126,hitop5,hitop261,hitop220,check_notatall,hitop15,hitop72,hitop140,hitop109,hitop197,hitop104,family_wealth,WHODAS_H2,WHODAS_H3,starttime,endtime,whichvisit
0,1250120682.0,1.123552,2,5,2,2,1,2,2,2,2,2,2,5,1,1964.0,1,1,5,1,6,21,12,2,1,1,1,1,2,1,1,1,2,2,2,2,2,2,2,__NA__,42,2,2,2,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,__NA__,1,1,1,2,2,2,2,2,2,2,2,2,__NA__,11,1,2,2,2,2,2,2,2,2,2,2,2,__NA__,1,2,9,9,9,9,__NA__,999,9,9,9,9,9,9,9,9,9,9,9,__NA__,999,2,1,1,2,3,3,4,4,2,4,1,1,2,4,3,1,1,1,1,1,1,2,4,1,4,4,1,1,4,2,4,4,1,4,4,3,4,4,4,4,4,1,1,4,1,2,4,4,4,4,4,1,3,1,1,3,1,3,4,2,3,1,4,1,1,4,1,4,4,4,4,1,3,4,3,1,1,1,1,4,1,4,2,1,3,2,1,4,2,1,4,2,3,2,1,4,3,2,4,1,1,4,2,1,4,2,3,1,2,4,4,4,4,4,1,1,4,2,1,4,4,1,1,2,4,1,1,2,80.0,0.0,14.0,2023-09-01T21:24:20.439,2023-09-01T21:44:16.602,recontactvisit
1,1250173970.0,0.954225,2,4,2,2,2,2,2,2,2,1,2,4,2,2004.0,2,2,3,5,8,17,1,2,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,No Data,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,No Data,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,No Data,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,No Data,-1,-1,-1,-1,-1,-1,No Data,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,No Data,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,No Data,No Data,No Data,No Data,No Data,recontactvisit
2,1250

In [71]:
df.shape

(310, 245)

In [72]:
df_helper.shape

(310, 245)

In [73]:
mydata_withrecontact = pd.concat([df, df_helper])

# Now CFA

In [74]:
null_mdl_strs = {}
mdl_strs = {}
for ss, df in item_lookup.groupby('subscale'):
    cln_ss = ss.replace(" ", "_").replace("/", "_").replace("-", "_")
    null_mdl_str = []
    psis = []
    taus = []
    items = []
    for itemn, item in enumerate(df.sort_values('id_ht').htid.values):
        psis.append(f'{item} ~~ c(psi{itemn+1}, psi{itemn+1})*{item}')
        taus.append(f'{item} ~ c(tau{itemn+1}, tau{itemn+1})*{item}')
        items.append(item)
    null_mdl_str = null_mdl_str + psis + taus
    null_mdl_strs[cln_ss] = null_mdl_str
    mdl_strs[cln_ss] = f'{cln_ss} =~' + ' + '.join(items)

In [75]:
# I don't understand how this null works, and I'm not sure where to look for it. Also, we end up not using it. Keeping it in this code but not going back to it later

In [76]:
mdl_strs

{'anhedonic_depression': 'anhedonic_depression =~hitop39 + hitop77 + hitop84 + hitop92 + hitop93 + hitop123 + hitop157 + hitop182 + hitop230 + hitop246',
 'anxious_worry': 'anxious_worry =~hitop20 + hitop34 + hitop89 + hitop203 + hitop248 + hitop265',
 'appetite_gain': 'appetite_gain =~hitop120 + hitop141 + hitop243 + hitop275',
 'appetite_loss': 'appetite_loss =~hitop109 + hitop280 + hitop283',
 'cognitive_problems': 'cognitive_problems =~hitop142 + hitop159 + hitop189',
 'hyposomnia': 'hyposomnia =~hitop5 + hitop66 + hitop99 + hitop181 + hitop231',
 'indecisiveness': 'indecisiveness =~hitop21 + hitop90 + hitop95',
 'insomnia': 'insomnia =~hitop160 + hitop254 + hitop261 + hitop268',
 'panic': 'panic =~hitop15 + hitop104 + hitop126 + hitop211 + hitop215 + hitop257',
 'separation_insecurity': 'separation_insecurity =~hitop40 + hitop50 + hitop69 + hitop81 + hitop113 + hitop136 + hitop151 + hitop197',
 'shame_guilt': 'shame_guilt =~hitop72 + hitop140 + hitop143 + hitop220',
 'situational_

In [77]:
# measures we can't work with because too few degrees of freedom:
# - 'appetite_loss': 'appetite_loss =~hitop109 + hitop280 + hitop283'
# - 'cognitive_problems': 'cognitive_problems =~hitop142 + hitop159 + hitop189',
# - 'indecisiveness': 'indecisiveness =~hitop21 + hitop90 + hitop95'

# here's the dictionary with items that *will* work
mdl_strs_new = {'anhedonic_depression': 'anhedonic_depression =~hitop39 + hitop77 + hitop84 + hitop92 + hitop93 + hitop123 + hitop157 + hitop182 + hitop230 + hitop246',
 'anxious_worry': 'anxious_worry =~hitop20 + hitop34 + hitop89 + hitop203 + hitop248 + hitop265',
 'appetite_gain': 'appetite_gain =~hitop120 + hitop141 + hitop243 + hitop275',
 'hyposomnia': 'hyposomnia =~hitop5 + hitop66 + hitop99 + hitop181 + hitop231',
 'insomnia': 'insomnia =~hitop160 + hitop254 + hitop261 + hitop268',
 'panic': 'panic =~hitop15 + hitop104 + hitop126 + hitop211 + hitop215 + hitop257',
 'separation_insecurity': 'separation_insecurity =~hitop40 + hitop50 + hitop69 + hitop81 + hitop113 + hitop136 + hitop151 + hitop197',
 'shame_guilt': 'shame_guilt =~hitop72 + hitop140 + hitop143 + hitop220',
 'situational_phobia': 'situational_phobia =~hitop16 + hitop165 + hitop225 + hitop247 + hitop278',
 'social_anxiety': 'social_anxiety =~hitop1 + hitop17 + hitop114 + hitop117 + hitop124 + hitop129 + hitop204 + hitop222 + hitop236 + hitop258',
 'well_being': 'well_being =~hitop9 + hitop23 + hitop54 + hitop106 + hitop149 + hitop200 + hitop244 + hitop245 + hitop250 + hitop281'}

In [78]:
wdata = 'valid_genpop' # valid_enriched or valid_genpop or enriched_withrecontact or genpop_withrecontact

# to load the data in R, let's save it as csv and then load it to R
# there may be a more elegant day to do this, but I don't know it...

if wdata == 'valid_genpop':
    mydata.to_csv('../data/CFA/mydata.csv')
    ro.r("rdata <- read.csv('../data/CFA/mydata.csv', header=TRUE)")
    group = 'whichdata'
elif wdata == 'valid_enriched':
    mydata_valid_enriched.to_csv('../data/CFA/mydata_valid_enriched.csv')
    ro.r("rdata <- read.csv('../data/CFA/mydata_valid_enriched.csv', header=TRUE)") 
    group = 'whichdata'
elif wdata == 'enriched_withrecontact':
    mydata_withrecontact.to_csv('../data/CFA/mydata_enriched_withrecontact.csv')
    ro.r("rdata <- read.csv('../data/CFA/mydata_enriched_withrecontact.csv', header=TRUE)") 
    group = 'whichvisit'
elif wdata == 'genpop_withrecontact':
    mydata_withrecontact.to_csv('../data/CFA/mydata_genpop_withrecontact.csv')
    ro.r("rdata <- read.csv('../data/CFA/mydata_genpop_withrecontact.csv', header=TRUE)")
    group = 'whichvisit'
ro.globalenv['group'] = group
rdata

NameError: name 'rdata' is not defined

In [79]:
lavaan = importr('lavaan')
semtools = importr('semTools')

In [80]:
rprint = ro.r('print')
summary = ro.r('summary')
def print_summary(obj):
    return rprint(summary(obj))

rngkind = "L'Ecuyer-CMRG"

def lv_to_df(lv):
    names = lv.names
    names = [nn.replace('.', '_') for nn in names]

    df = pd.DataFrame(lv, index=names, columns=['val'])
    return df

def permmeaseq_to_df(permmeaseq):
    anova_res = lv_to_df(permmeaseq.slots['ANOVA'])
    anova_res.index = [ii+'_anova' for ii in anova_res.index]
    afi_dif = lv_to_df(permmeaseq.slots['AFI.obs'])
    afi_dif.index = [ii+'_diff' for ii in afi_dif.index]
    afi_ps = lv_to_df(permmeaseq.slots['AFI.pval'])
    afi_ps.index = [ii+'_p' for ii in afi_ps.index]
    return pd.concat([anova_res, afi_dif, afi_ps])

In [81]:
num_iter = 1000

for cln_ss, mdl in mdl_strs_new.items():
    #if cln_ss == 'anhedonic_depression':
    print("\n\n\n================================")
    print("================================")
    print(cln_ss.upper())
    print(mdl)
    print("================================")
    print("================================")
    print("\nCONFIG\n")
    ro.globalenv[cln_ss+'_mdl'] = mdl_strs_new[cln_ss]
    fit_config = ro.r(f'cfa({cln_ss}_mdl, data = rdata, group = group, estimator = "WLSMV")')
    ro.globalenv['fit_config'] = fit_config
    #ro.r('print(summary(fit_config, fit.measures=TRUE))')
    ro.r("myoutputconfig <- capture.output(summary(fit_config, fit.measures=TRUE))")
    #ro.r("print(myoutputconfig)")
    ro.r("stringid_cfi <- grep(\"Robust.*CFI\", myoutputconfig)")
    ro.r("stringid_tli <- grep(\"Robust.*TLI\", myoutputconfig)")
    ro.r("stringid_rmsea <- grep(\"Robust.*RMSEA\", myoutputconfig)")
    ro.r("print(myoutputconfig[stringid_cfi])")
    ro.r("print(myoutputconfig[stringid_tli])")
    ro.r("print(myoutputconfig[stringid_rmsea])")
    print("----------------")
    out_config = semtools.permuteMeasEq(nPermute=num_iter, # 100 for now, 1000 for the "real" run
                                            con=fit_config, # In the case of testing configural invariance when modelType = "mgcfa", con is the configural model (implicitly, the unconstrained model is the saturated model, so use the defaults uncon = NULL and param = NULL).
                                            parallelType="multicore", ncpus=4)
    #_ = print_summary(out_config) 
    ro.globalenv['out_config'] = out_config
    #ro.r('print(summary(out_config))')
    ro.r("myoutput <- capture.output(summary(out_config))") 
    #ro.r("print(class(myoutput))")        
    ro.r("stringid <- grep(\"chisq\", myoutput)")
    ro.r("print(myoutput[stringid-1])")
    ro.r("print(myoutput[stringid])")
    print("================")
    print("\nMETRIC\n")
    fit_metric = ro.r(f'cfa({cln_ss}_mdl, data = rdata, group = group, estimator = "WLSMV", group.equal="loadings")')
    ro.globalenv['fit_metric'] = fit_metric
    #ro.r('print(summary(fit_metric, fit.measures=TRUE))')
    #ro.r("myoutputmetric <- capture.output(summary(fit_metric, fit.measures=TRUE))")
    print("--------------")
    out_metric = semtools.permuteMeasEq(nPermute=num_iter, 
                                        uncon=fit_config, 
                                        con=fit_metric, 
                                        param="loadings",
                                        parallelType="multicore", ncpus=4)
    #_ = print_summary(out_metric)
    ro.globalenv['out_metric'] = out_metric
    ro.r("myoutputmetric <- capture.output(summary(out_metric))") 
    ro.r("stringid <- grep(\"chisq\", myoutputmetric)")
    ro.r("print(myoutputmetric[stringid-1])")
    ro.r("print(myoutputmetric[stringid])")
    # significant:
    ro.r("stringidsig <- grep(\"may differ between Groups\", myoutputmetric)")
    ro.r("print(myoutputmetric[stringidsig])")
    print("================")
    print("\nSCALAR\n")
    fit_scalar = ro.r(f'cfa({cln_ss}_mdl, data = rdata, group = group, estimator = "WLSMV", group.equal=c("loadings", "intercepts"))')
    ro.globalenv['fit_scalar'] = fit_scalar
    #ro.r('print(summary(fit_scalar, fit.measures=TRUE))')
    print("--------------")
    out_scalar = semtools.permuteMeasEq(nPermute=num_iter, 
                                    uncon=fit_metric, 
                                    con=fit_scalar, 
                                    param=ro.StrVector(["loadings","intercepts"]),
                                    parallelType="multicore", 
                                    ncpus=4)
    #_ = print_summary(out_scalar)
    ro.globalenv['out_scalar'] = out_scalar
    ro.r("myoutputscalar <- capture.output(summary(out_scalar))") 
    ro.r("stringid <- grep(\"chisq\", myoutputscalar)")
    ro.r("print(myoutputscalar[stringid-1])")
    ro.r("print(myoutputscalar[stringid])")
    # significant:
    ro.r("stringidsig <- grep(\"may differ between Groups\", myoutputscalar)")
    ro.r("print(myoutputscalar[stringidsig])") 
    print("================")
    print("\nSTRICT\n")
    fit_strict = ro.r(f'cfa({cln_ss}_mdl, data = rdata, group = group, estimator = "WLSMV", group.equal=c("loadings", "intercepts", "residuals"))')
    ro.globalenv['fit_strict'] = fit_strict
    #ro.r('print(summary(fit_strict, fit.measures=TRUE))')
    print("--------------")
    out_strict = semtools.permuteMeasEq(nPermute=num_iter, 
                                        uncon=fit_scalar, 
                                        con=fit_strict, 
                                        param=ro.StrVector(["loadings","intercepts", "residuals"]),
                                        parallelType="multicore", 
                                        ncpus=4)
    #_ = print_summary(out_strict)
    ro.globalenv['out_strict'] = out_strict
    ro.r("myoutputstrict <- capture.output(summary(out_strict))") 
    ro.r("stringid <- grep(\"chisq\", myoutputstrict)")
    ro.r("print(myoutputstrict[stringid-1])")
    ro.r("print(myoutputstrict[stringid])")
    # significant:
    ro.r("stringidsig <- grep(\"may differ between Groups\", myoutputstrict)")
    ro.r("print(myoutputstrict[stringidsig])") 
print("DONE")




ANHEDONIC_DEPRESSION
anhedonic_depression =~hitop39 + hitop77 + hitop84 + hitop92 + hitop93 + hitop123 + hitop157 + hitop182 + hitop230 + hitop246

CONFIG

[1] "  Robust Comparative Fit Index (CFI)                         0.997"
[1] "  Robust Tucker-Lewis Index (TLI)                            0.996"
[1] "  Robust RMSEA                                               0.047"
[2] "  P-value H_0: Robust RMSEA <= 0.050                         0.804"
[3] "  P-value H_0: Robust RMSEA >= 0.080                         0.000"


R[write to console]: Your RNGkind() was changed from Mersenne-Twister to L'Ecuyer-CMRG, which is required for reproducibility  in parallel jobs.  Your RNGkind() has been returned to Mersenne-Twister but the seed has not been set.  The state of your previous RNG is saved in the slot  named 'oldSeed', if you want to restore it using  the syntax:
.Random.seed[-1] <- permuteMeasEqObject@oldSeed[-1]

R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




----------------
[1] "      AFI.Difference p.value"
[1] "chisq         96.755       0"


R[write to console]: Your RNGkind() was changed from Mersenne-Twister to L'Ecuyer-CMRG, which is required for reproducibility  in parallel jobs.  Your RNGkind() has been returned to Mersenne-Twister but the seed has not been set.  The state of your previous RNG is saved in the slot  named 'oldSeed', if you want to restore it using  the syntax:
.Random.seed[-1] <- permuteMeasEqObject@oldSeed[-1]

R[write to console]: No AFIs were selected, so only chi-squared will be permuted.





METRIC

--------------


R[write to console]: 

R[write to console]: Warning message:

R[write to console]: In lavaan::lavTestScore(dots$con) :
R[write to console]: 
 
R[write to console]:  lavaan WARNING: se is not `standard'; not implemented yet; falling back to ordinary score test



RRuntimeError: 

# ------ BELOW ARE INDIVIDUAL EXPERIMENTS ------- #

## --> Ablation experiments

In [ ]:
'''
appetite gain and insomnia don't need a subset of invariant items, they pass all invariance tests
experimenting with the following:

 'anhedonic_depression': 'anhedonic_depression =~hitop39 + hitop77 + hitop84 + hitop92 + hitop93 + hitop123 + hitop157 + hitop182 + hitop230 + hitop246',
 'anxious_worry': 'anxious_worry =~hitop20 + hitop34 + hitop89 + hitop203 + hitop248 + hitop265',
 'hyposomnia': 'hyposomnia =~hitop5 + hitop66 + hitop99 + hitop181 + hitop231',
 'panic': 'panic =~hitop15 + hitop104 + hitop126 + hitop211 + hitop215 + hitop257',
 'separation_insecurity': 'separation_insecurity =~hitop40 + hitop50 + hitop69 + hitop81 + hitop113 + hitop136 + hitop151 + hitop197',
 'shame_guilt': 'shame_guilt =~hitop72 + hitop140 + hitop143 + hitop220',
 'situational_phobia': 'situational_phobia =~hitop16 + hitop165 + hitop225 + hitop247 + hitop278',
 'social_anxiety': 'social_anxiety =~hitop1 + hitop17 + hitop114 + hitop117 + hitop124 + hitop129 + hitop204 + hitop222 + hitop236 + hitop258',
 'well_being': 'well_being =~hitop9 + hitop23 + hitop54 + hitop106 + hitop149 + hitop200 + hitop244 + hitop245 + hitop250 + hitop281'}
 '''

In [357]:
num_iter = 1000
hitop_scale = 'well_being' 
scales = ['metric', 'scalar', 'strict'] # configural is performed by default; the order is --> metric, scalar, strict

for cln_ss, mdl in mdl_strs_new.items():
    if cln_ss == hitop_scale:
        print("\n\n\n================================")
        print("================================")
        print(cln_ss.upper())
        #mdl = 'situational_phobia =~hitop16 + hitop165 + hitop225 + hitop247'
        print(mdl)
        print("================================")
        print("================================")
        print("\nCONFIG\n")
        #ro.globalenv[cln_ss+'_mdl'] = mdl_strs_new[cln_ss]
        ro.globalenv[cln_ss+'_mdl'] = mdl
        fit_config = ro.r(f'cfa({cln_ss}_mdl, data = rdata, group = "whichdata", estimator = "WLSMV")')
        ro.globalenv['fit_config'] = fit_config
        #ro.r('print(summary(fit_config, fit.measures=TRUE))')
        ro.r("myoutputconfig <- capture.output(summary(fit_config, fit.measures=TRUE))")
        #ro.r("print(myoutputconfig)")
        ro.r("stringid_cfi <- grep(\"Robust.*CFI\", myoutputconfig)")
        ro.r("stringid_tli <- grep(\"Robust.*TLI\", myoutputconfig)")
        ro.r("stringid_rmsea <- grep(\"Robust.*RMSEA\", myoutputconfig)")
        ro.r("print(myoutputconfig[stringid_cfi])")
        ro.r("print(myoutputconfig[stringid_tli])")
        ro.r("print(myoutputconfig[stringid_rmsea])")
        print("----------------")
        out_config = semtools.permuteMeasEq(nPermute=num_iter, # 100 for now, 1000 for the "real" run
                                                con=fit_config, # In the case of testing configural invariance when modelType = "mgcfa", con is the configural model (implicitly, the unconstrained model is the saturated model, so use the defaults uncon = NULL and param = NULL).
                                                parallelType="multicore", ncpus=4)
        #_ = print_summary(out_config) 
        ro.globalenv['out_config'] = out_config
        #ro.r('print(summary(out_config))')
        ro.r("myoutput <- capture.output(summary(out_config))") 
        #ro.r("print(class(myoutput))")        
        ro.r("stringid <- grep(\"chisq\", myoutput)")
        ro.r("print(myoutput[stringid-1])")
        ro.r("print(myoutput[stringid])")
        if 'metric' in scales:
            print("================")
            print("\nMETRIC\n")
            fit_metric = ro.r(f'cfa({cln_ss}_mdl, data = rdata, group = "whichdata", estimator = "WLSMV", group.equal="loadings")')
            ro.globalenv['fit_metric'] = fit_metric
            #ro.r('print(summary(fit_metric, fit.measures=TRUE))')
            #ro.r("myoutputmetric <- capture.output(summary(fit_metric, fit.measures=TRUE))")
            print("--------------")
            out_metric = semtools.permuteMeasEq(nPermute=num_iter, 
                                                uncon=fit_config, 
                                                con=fit_metric, 
                                                param="loadings",
                                                parallelType="multicore", ncpus=4)
            #_ = print_summary(out_metric)
            ro.globalenv['out_metric'] = out_metric
            ro.r("myoutputmetric <- capture.output(summary(out_metric))") 
            ro.r("stringid <- grep(\"chisq\", myoutputmetric)")
            ro.r("print(myoutputmetric[stringid-1])")
            ro.r("print(myoutputmetric[stringid])")
            # significant:
            ro.r("stringidsig <- grep(\"may differ between Groups\", myoutputmetric)")
            ro.r("print(myoutputmetric[stringidsig])")
            print("================")
            if 'scalar' in scales:
                print("\nSCALAR\n")
                fit_scalar = ro.r(f'cfa({cln_ss}_mdl, data = rdata, group = "whichdata", estimator = "WLSMV", group.equal=c("loadings", "intercepts"))')
                ro.globalenv['fit_scalar'] = fit_scalar
                #ro.r('print(summary(fit_scalar, fit.measures=TRUE))')
                print("--------------")
                out_scalar = semtools.permuteMeasEq(nPermute=num_iter, 
                                                uncon=fit_metric, 
                                                con=fit_scalar, 
                                                param=ro.StrVector(["loadings","intercepts"]),
                                                parallelType="multicore", 
                                                ncpus=4)
                #_ = print_summary(out_scalar)
                ro.globalenv['out_scalar'] = out_scalar
                ro.r("myoutputscalar <- capture.output(summary(out_scalar))") 
                ro.r("stringid <- grep(\"chisq\", myoutputscalar)")
                ro.r("print(myoutputscalar[stringid-1])")
                ro.r("print(myoutputscalar[stringid])")
                # significant:
                ro.r("stringidsig <- grep(\"may differ between Groups\", myoutputscalar)")
                ro.r("print(myoutputscalar[stringidsig])") 
                print("================")
                if 'strict' in scales:
                    print("\nSTRICT\n")
                    fit_strict = ro.r(f'cfa({cln_ss}_mdl, data = rdata, group = "whichdata", estimator = "WLSMV", group.equal=c("loadings", "intercepts", "residuals"))')
                    ro.globalenv['fit_strict'] = fit_strict
                    #ro.r('print(summary(fit_strict, fit.measures=TRUE))')
                    print("--------------")
                    out_strict = semtools.permuteMeasEq(nPermute=num_iter, 
                                                        uncon=fit_scalar, 
                                                        con=fit_strict, 
                                                        param=ro.StrVector(["loadings","intercepts", "residuals"]),
                                                        parallelType="multicore", 
                                                        ncpus=4)
                    #_ = print_summary(out_strict)
                    ro.globalenv['out_strict'] = out_strict
                    ro.r("myoutputstrict <- capture.output(summary(out_strict))") 
                    ro.r("stringid <- grep(\"chisq\", myoutputstrict)")
                    ro.r("print(myoutputstrict[stringid-1])")
                    ro.r("print(myoutputstrict[stringid])")
                    # significant:
                    ro.r("stringidsig <- grep(\"may differ between Groups\", myoutputstrict)")
                    ro.r("print(myoutputstrict[stringidsig])") 
print("DONE")






WELL_BEING
well_being =~hitop9 + hitop23 + hitop54 + hitop106 + hitop149 + hitop200 + hitop244 + hitop245 + hitop250 + hitop281

CONFIG

[1] "  Robust Comparative Fit Index (CFI)                         1.000"
[1] "  Robust Tucker-Lewis Index (TLI)                            1.000"
[1] "  Robust RMSEA                                               0.043"
[2] "  P-value H_0: Robust RMSEA <= 0.050                         0.963"
[3] "  P-value H_0: Robust RMSEA >= 0.080                         0.000"


R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




----------------
[1] "      AFI.Difference p.value"
[1] "chisq         71.273       0"


R[write to console]: No AFIs were selected, so only chi-squared will be permuted.





METRIC

--------------
[1] "      AFI.Difference p.value"
[1] "chisq         38.689   0.029"
character(0)


R[write to console]: No AFIs were selected, so only chi-squared will be permuted.





SCALAR

--------------
[1] "      AFI.Difference p.value"
[1] "chisq         78.547       0"
[1] "Parameter 'hitop9~1' may differ between Groups 'validation' and 'enriched'." 
[2] "Parameter 'hitop23~1' may differ between Groups 'validation' and 'enriched'."


R[write to console]: No AFIs were selected, so only chi-squared will be permuted.





STRICT

--------------
[1] "      AFI.Difference p.value"
[1] "chisq         13.027   0.074"
[1] "Parameter 'hitop9~1' may differ between Groups 'validation' and 'enriched'." 
[2] "Parameter 'hitop23~1' may differ between Groups 'validation' and 'enriched'."
DONE


# Metric only

In [524]:
num_iter = 1000
hitop_scale = 'situational_phobia' 
scales = ['metric'] # configural is performed by default; the order is --> metric, scalar, strict

for cln_ss, mdl in mdl_strs_new.items():
    if cln_ss == hitop_scale:
        print("\n\n\n================================")
        print("================================")
        print(cln_ss.upper())
        mdl = 'situational_phobia =~hitop16 + hitop165 + hitop247 + hitop278'
        print(mdl)
        print("================================")
        print("================================")
        print("\nCONFIG\n")
        #ro.globalenv[cln_ss+'_mdl'] = mdl_strs_new[cln_ss]
        ro.globalenv[cln_ss+'_mdl'] = mdl
        fit_config = ro.r(f'cfa({cln_ss}_mdl, data = rdata, group = "whichdata", estimator = "WLSMV")')
        ro.globalenv['fit_config'] = fit_config
        #ro.r('print(summary(fit_config, fit.measures=TRUE))')
        ro.r("myoutputconfig <- capture.output(summary(fit_config, fit.measures=TRUE))")
        #ro.r("print(myoutputconfig)")
        ro.r("stringid_cfi <- grep(\"Robust.*CFI\", myoutputconfig)")
        ro.r("stringid_tli <- grep(\"Robust.*TLI\", myoutputconfig)")
        ro.r("stringid_rmsea <- grep(\"Robust.*RMSEA\", myoutputconfig)")
        ro.r("print(myoutputconfig[stringid_cfi])")
        ro.r("print(myoutputconfig[stringid_tli])")
        ro.r("print(myoutputconfig[stringid_rmsea])")
        print("----------------")
        out_config = semtools.permuteMeasEq(nPermute=num_iter, # 100 for now, 1000 for the "real" run
                                                con=fit_config, # In the case of testing configural invariance when modelType = "mgcfa", con is the configural model (implicitly, the unconstrained model is the saturated model, so use the defaults uncon = NULL and param = NULL).
                                                parallelType="multicore", ncpus=4)
        #_ = print_summary(out_config) 
        ro.globalenv['out_config'] = out_config
        #ro.r('print(summary(out_config))')
        ro.r("myoutput <- capture.output(summary(out_config))") 
        #ro.r("print(class(myoutput))")        
        ro.r("stringid <- grep(\"chisq\", myoutput)")
        ro.r("print(myoutput[stringid-1])")
        ro.r("print(myoutput[stringid])")
        if 'metric' in scales:
            print("================")
            print("\nMETRIC\n")
            fit_metric = ro.r(f'cfa({cln_ss}_mdl, data = rdata, group = "whichdata", estimator = "WLSMV", group.equal="loadings")')
            ro.globalenv['fit_metric'] = fit_metric
            #ro.r('print(summary(fit_metric, fit.measures=TRUE))')
            #ro.r("myoutputmetric <- capture.output(summary(fit_metric, fit.measures=TRUE))")
            print("--------------")
            out_metric = semtools.permuteMeasEq(nPermute=num_iter, 
                                                uncon=fit_config, 
                                                con=fit_metric, 
                                                param="loadings",
                                                parallelType="multicore", ncpus=4)
            #_ = print_summary(out_metric)
            ro.globalenv['out_metric'] = out_metric
            ro.r("myoutputmetric <- capture.output(summary(out_metric))") 
            ro.r("stringid <- grep(\"chisq\", myoutputmetric)")
            ro.r("print(myoutputmetric[stringid-1])")
            ro.r("print(myoutputmetric[stringid])")
            # significant:
            ro.r("stringidsig <- grep(\"may differ between Groups\", myoutputmetric)")
            ro.r("print(myoutputmetric[stringidsig])")
            print("================")
print('DONE')




SITUATIONAL_PHOBIA
situational_phobia =~hitop16 + hitop165 + hitop247 + hitop278

CONFIG

[1] "  Robust Comparative Fit Index (CFI)                         0.997"
[1] "  Robust Tucker-Lewis Index (TLI)                            0.990"
[1] "  Robust RMSEA                                               0.044"
[2] "  P-value H_0: Robust RMSEA <= 0.050                         0.609"
[3] "  P-value H_0: Robust RMSEA >= 0.080                         0.012"


R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




----------------
[1] "      AFI.Difference p.value"
[1] "chisq          5.317    0.53"


R[write to console]: No AFIs were selected, so only chi-squared will be permuted.





METRIC

--------------
[1] "      AFI.Difference p.value"
[1] "chisq          1.314   0.627"
character(0)
DONE


In [528]:
items_to_check = ['9','106','149','200','244','250','281']
for i in items_to_check:
    a = repr(item_lookup.loc[item_lookup['htid'] == 'hitop'+i]['htid']).split(' ', 1)[1]
    aa = str(a).split('\n')[0]
    b = repr(item_lookup.loc[item_lookup['htid'] == 'hitop'+i]['item']).split(' ', 1)[1]
    bb = str(b).split('\n')[0]
    print(aa+' : '+bb)

   hitop9 :    I felt like I was having a lot of fun.
   hitop106 :    I was proud of myself.
   hitop149 :    I felt good about myself.
   hitop200 :    I found compliments very encouraging.
   hitop244 :    I felt optimistic.
   hitop250 :    I felt like I had a lot of interesting things to do.
   hitop281 :    I felt that I had a lot to look forward to.


In [ ]:
[1] "Parameter 'well_being=~hitop23' may differ between Groups 'validation' and 'genpop'." 
[2] "Parameter 'well_being=~hitop54' may differ between Groups 'validation' and 'genpop'." 
[3] "Parameter 'well_being=~hitop245' may differ between Groups 'validation' and 'genpop'."

Removing them

well_being =~hitop9 + hitop106 + hitop149 + hitop200 + hitop244 + hitop250 + hitop281


## 